In [22]:
from pulp import *
import time
import pandas as pd
import plotly as py

# This is the block you should make changes
# Only change things to the right of the '=' sign. 
# Note that the '#' symbol means a comment-- when the code runs, it ignores this.
# Here you can change the scnario name, number of warehouses, the distance bands, and whether you draw the maps

# To Run, go to the menu item Cell and select 'Run All'
# To see the results of the run: 
#      Go to the directory where you put this file and you'll see two output files
#      One output file will be the summary and one will have details
#      And, tabs will open if you decide to map the input and output

# You should change the name to the right of the '=' sign. This names each scenario. 
# If you don't change it, you'll overwrite the files when you run.  Don't put spaces in the name
# And, kep the single quotes in the name.  

scenario_name = 'scenario_1'  #<<<----- you can change this name

# Change the number of warehouses you would like to select.  This number should be an integer

number_of_whs = 3  #<<<------ you can change this number

# Change the distance bands, if you'd like. The band numbers represent the distance up to.  These should be increasing.
# You cannot add or subtract bands.

distance_band_1 = 200  #<<<----- you can change these numbers
distance_band_2 = 400
distance_band_3 = 800
distance_band_4 = 1600

# If you don't want either the input or output map to draw, type False with a capital F instead of True

input_map = True   #<<<----- you can change these from True to False
output_map = True

# Transportation Rate per ton-mile
ob_trans_cost = 0.12   #outbound transportation cost per unit of ton-mile
ic_trans_cost = 0.07   #inter company transportation cost per ton-mile
ob_min_trans = 10      #outbound min charge per ton
ic_min_trans = 10      #inter company min charge per ton

#{('Plant ID', 'Product ID'): Capacity}  Each row is a plant. You can change the number after the two  numbers in parenthesis.  
# You want to put in a large number, like 210000000 when a plant can make a product.  
# For example, the number in the first row after "(1,2):" is the amount of product 2 that Plant 1 can make.
plant_product_info = {(1, 1): 210000000 , (1, 2): 210000000, (1, 3): 0, (1, 4): 0,
                      (2, 1): 0, (2, 2): 210000000, (2, 3): 210000000, (2, 4): 0,
                      (3, 1): 0, (3, 2): 0, (3, 3): 210000000, (3, 4): 210000000,
                      (4, 1): 210000000, (4, 2): 0, (4, 3): 0, (4, 4): 210000000}

#************This is where you change the status of a warehouse************
# meaning you can force it to be open or force it closed.
# You want to changet the two numbers after the name.  Don't delete or change the commas
# it should be 0,1 to let the solver pick, 1,1 to force it to be used, or 0,0 to force it closed.  1,0 is an error- don't use that

wh_status = {1: ('Los Angeles', 0,1),
    2: ('New York City', 0,1),
    3: ('Chicago', 0,1),
    7: ('Houston', 0,1), 
    8: ('Atlanta', 0,1), 
    9: ('Dallas', 0,1), 
    11: ('Phoenix', 0,1), 
    14: ('New York', 0,1), 
    15: ('St. Louis', 0,1), 
    16: ('Baltimore', 0,1), 
    17: ('Pittsburgh', 0,1), 
    19: ('Seattle', 0,1), 
    25: ('Denver', 0,1), 
    27: ('Kansas City', 0,1), 
    28: ('San Francisco', 0,1), 
    38: ('Orlando', 0,1), 
    39: ('Columbus', 0,1), 
    43: ('New Orleans',0,1), 
    44: ('Las Vegas', 0,1), 
    45: ('Salt Lake City', 0,1), 
    52: ('Memphis', 0,1), 
    55: ('Raleigh', 0,1), 
    57: ('Oklahoma City', 0,1), 
    60: ('Louisville', 0,1), 
    99: ('Columbia', 0,1)}



# To Run, go to the menu item Cell and select 'Run All'


In [23]:
#getting input data
def get_data():
    #{'Customer ID': ('Name', 'Active', 'City', 'State', 'Zip Code', 'Country or Region', 'Latitude', 'Longitude')}
    customers = {1: ('Los Angeles', True, 'Los Angeles', 'CA', 90001, 'USA', 33.974044, -118.248849), 2: ('New York City', True, 'New York City', 'NY', 10001, 'USA', 40.75012, -73.997846), 3: ('Chicago', True, 'Chicago', 'IL', 60602, 'USA', 41.88331, -87.624713), 4: ('Boston', True, 'Boston', 'MA', 2101, 'USA', 42.36097, -71.05344), 5: ('Washington', True, 'Washington', 'DC', 20001, 'USA', 38.910004, -77.017989), 6: ('Detroit', True, 'Detroit', 'MI', 48201, 'USA', 42.346577, -83.059731), 7: ('Houston', True, 'Houston', 'TX', 77001, 'USA', 29.757317, -95.365005), 8: ('Atlanta', True, 'Atlanta', 'GA', 30301, 'USA', 33.753693, -84.389544), 9: ('Dallas', True, 'Dallas', 'TX', 75201, 'USA', 32.787642, -96.799525), 10: ('Riverside', True, 'Riverside', 'CA', 92501, 'USA', 33.994941, -117.372528), 11: ('Phoenix', True, 'Phoenix', 'AZ', 85001, 'USA', 33.451015, -112.068554), 12: ('Minneapolis', True, 'Minneapolis', 'MN', 55401, 'USA', 44.985775, -93.270165), 13: ('Orange', True, 'Orange', 'CA', 92850, 'USA', 33.844425, -117.951222), 14: ('Nassau-Suffolk', True, 'Nassau-Suffolk', 'NY', 12123, 'USA', 42.526891, -73.609283), 15: ('St. Louis', True, 'St. Louis', 'MO', 63101, 'USA', 38.631358, -90.192246), 16: ('Baltimore', True, 'Baltimore', 'MD', 21201, 'USA', 39.294398, -76.622747), 17: ('Pittsburgh', True, 'Pittsburgh', 'PA', 15201, 'USA', 40.474802, -79.95449), 18: ('San Diego', True, 'San Diego', 'CA', 92101, 'USA', 32.724693, -117.164885), 19: ('Seattle', True, 'Seattle', 'WA', 98101, 'USA', 47.611601, -122.333038), 20: ('Oakland', True, 'Oakland', 'CA', 94601, 'USA', 37.777255, -122.217025), 21: ('Tampa', True, 'Tampa', 'FL', 33601, 'USA', 27.95454, -82.45734), 22: ('Cleveland', True, 'Cleveland', 'OH', 44101, 'USA', 41.491407, -81.671411), 23: ('Miami', True, 'Miami', 'FL', 33101, 'USA', 25.778271, -80.198105), 24: ('Newark', True, 'Newark', 'NJ', 7101, 'USA', 40.735652, -74.17309), 25: ('Denver', True, 'Denver', 'CO', 80201, 'USA', 39.75071, -104.996225), 26: ('Portland', True, 'Portland', 'OR', 97201, 'USA', 45.498251, -122.692496), 27: ('Kansas City', True, 'Kansas City', 'MO', 64101, 'USA', 39.103883, -94.600613), 28: ('San Francisco', True, 'San Francisco', 'CA', 94102, 'USA', 37.779887, -122.418066), 29: ('New Haven', True, 'New Haven', 'CT', 6501, 'USA', 41.30758, -72.92471), 30: ('San Jose', True, 'San Jose', 'CA', 95101, 'USA', 37.386691, -121.89705), 31: ('Cincinnati', True, 'Cincinnati', 'OH', 45201, 'USA', 39.10663, -84.49974), 32: ('Fort Worth', True, 'Fort Worth', 'TX', 76101, 'USA', 32.75826, -97.32787), 33: ('Norfolk', True, 'Norfolk', 'VA', 23501, 'USA', 36.852105, -76.292507), 34: ('San Antonio', True, 'San Antonio', 'TX', 78201, 'USA', 29.469324, -98.527988), 35: ('Indianapolis', True, 'Indianapolis', 'IN', 46201, 'USA', 39.77422, -86.109309), 36: ('Sacramento', True, 'Sacramento', 'CA', 94203, 'USA', 38.585076, -121.491186), 37: ('Fort Lauderdale', True, 'Fort Lauderdale', 'FL', 33301, 'USA', 26.121066, -80.127818), 38: ('Orlando', True, 'Orlando', 'FL', 32801, 'USA', 28.541627, -81.37397), 39: ('Columbus', True, 'Columbus', 'OH', 43085, 'USA', 40.099395, -83.016823), 40: ('Milwaukee', True, 'Milwaukee', 'WI', 53201, 'USA', 43.037602, -87.916246), 41: ('Charlotte', True, 'Charlotte', 'NC', 28201, 'USA', 35.194361, -80.826397), 42: ('Bergen', True, 'Bergen', 'NJ', 7055, 'USA', 40.857466, -74.128618), 43: ('New Orleans', True, 'New Orleans', 'LA', 70112, 'USA', 29.956664, -90.077506), 44: ('Las Vegas', True, 'Las Vegas', 'NV', 89101, 'USA', 36.17269, -115.121117), 45: ('Salt Lake City', True, 'Salt Lake City', 'UT', 84101, 'USA', 40.755074, -111.89849), 46: ('Buffalo', True, 'Buffalo', 'NY', 14201, 'USA', 42.896219, -78.884649), 47: ('Greensboro', True, 'Greensboro', 'NC', 27401, 'USA', 36.070061, -79.766346), 48: ('Nashville', True, 'Nashville', 'TN', 37201, 'USA', 36.164003, -86.7745), 49: ('Hartford', True, 'Hartford', 'CT', 6101, 'USA', 41.789, -72.666218), 50: ('Middlesex', True, 'Middlesex', 'NJ', 8846, 'USA', 40.574084, -74.5012), 51: ('Rochester', True, 'Rochester', 'NY', 14601, 'USA', 43.1683, -77.60262), 52: ('Memphis', True, 'Memphis', 'TN', 37501, 'USA', 35.033731, -89.934319), 53: ('Monmouth', True, 'Monmouth', 'NJ', 7750, 'USA', 40.333745, -73.982438), 54: ('Austin', True, 'Austin', 'TX', 78701, 'USA', 30.270515, -97.74218), 55: ('Raleigh', True, 'Raleigh', 'NC', 27601, 'USA', 35.773856, -78.634051), 56: ('Jacksonville', True, 'Jacksonville', 'FL', 32099, 'USA', 30.340415, -81.830904), 57: ('Oklahoma City', True, 'Oklahoma City', 'OK', 73101, 'USA', 35.467763, -97.520977), 58: ('Grand Rapids', True, 'Grand Rapids', 'MI', 49501, 'USA', 42.965899, -85.654579), 59: ('West Palm Beach', True, 'West Palm Beach', 'FL', 33427, 'USA', 26.345694, -80.084229), 60: ('Louisville', True, 'Louisville', 'KY', 40201, 'USA', 38.253601, -85.768124), 61: ('Dayton', True, 'Dayton', 'OH', 45401, 'USA', 39.758119, -84.1879), 62: ('Richmond', True, 'Richmond', 'VA', 23218, 'USA', 37.538812, -77.436395), 63: ('Philadelphia', True, 'Philadelphia', 'PA', 19101, 'USA', 39.961558, -75.199624), 64: ('Greenville', True, 'Greenville', 'SC', 29601, 'USA', 34.847881, -82.403745), 65: ('Providence', True, 'Providence', 'RI', 2901, 'USA', 41.817904, -71.409316), 66: ('Birmingham', True, 'Birmingham', 'AL', 35201, 'USA', 33.518634, -86.808862), 67: ('Albany', True, 'Albany', 'NY', 12201, 'USA', 42.65287, -73.74868), 68: ('Fresno', True, 'Fresno', 'CA', 93701, 'USA', 36.749611, -119.786244), 69: ('Florence', True, 'Florence', 'SC', 29501, 'USA', 34.198188, -79.826641), 70: ('Tucson', True, 'Tucson', 'AZ', 85701, 'USA', 32.216256, -110.970728), 71: ('Tulsa', True, 'Tulsa', 'OK', 74101, 'USA', 36.149273, -95.975637), 72: ('Syracuse', True, 'Syracuse', 'NY', 13201, 'USA', 43.04335, -76.15084), 73: ('Ventura', True, 'Ventura', 'CA', 93001, 'USA', 34.345883, -119.328846), 74: ('El Paso', True, 'El Paso', 'TX', 79901, 'USA', 31.760613, -106.47948), 75: ('Omaha', True, 'Omaha', 'NE', 68101, 'USA', 41.263251, -95.932956), 76: ('Akron', True, 'Akron', 'OH', 44301, 'USA', 41.043696, -81.524301), 77: ('Albuquerque', True, 'Albuquerque', 'NM', 87101, 'USA', 35.082729, -106.646276), 78: ('Tacoma', True, 'Tacoma', 'WA', 98401, 'USA', 47.253432, -122.435514), 79: ('Knoxville', True, 'Knoxville', 'TN', 37901, 'USA', 35.946445, -84.022857), 80: ('Bakersfield', True, 'Bakersfield', 'CA', 93301, 'USA', 35.384352, -119.019342), 81: ('Gary', True, 'Gary', 'IN', 46401, 'USA', 41.586162, -87.319082), 82: ('Scranton', True, 'Scranton', 'PA', 18501, 'USA', 41.358057, -75.70249), 83: ('Harrisburg', True, 'Harrisburg', 'PA', 17101, 'USA', 40.264554, -76.880588), 84: ('Allentown', True, 'Allentown', 'PA', 18101, 'USA', 40.602812, -75.470433), 85: ('Toledo', True, 'Toledo', 'OH', 43601, 'USA', 41.645022, -83.548348), 86: ('Youngstown', True, 'Youngstown', 'OH', 44501, 'USA', 41.099765, -80.64936), 87: ('Springfield', True, 'Springfield', 'MA', 1101, 'USA', 42.105997, -72.597915), 88: ('Baton Rouge', True, 'Baton Rouge', 'LA', 70801, 'USA', 30.449314, -91.187593), 89: ('Wilmington', True, 'Wilmington', 'DE', 19801, 'USA', 39.728445, -75.537257), 90: ('Little Rock', True, 'Little Rock', 'AR', 72201, 'USA', 34.747348, -92.278932), 91: ('Jersey City', True, 'Jersey City', 'NJ', 7097, 'USA', 40.751783, -74.05392), 92: ('Stockton', True, 'Stockton', 'CA', 95201, 'USA', 37.930437, -121.436022), 93: ('Ann Arbor', True, 'Ann Arbor', 'MI', 48103, 'USA', 42.264476, -83.839628), 94: ('Sarasota', True, 'Sarasota', 'FL', 34230, 'USA', 27.337217, -82.51411), 95: ('Wichita', True, 'Wichita', 'KS', 67201, 'USA', 37.686354, -97.334936), 96: ('Mobile', True, 'Mobile', 'AL', 36601, 'USA', 30.692504, -88.04323), 97: ('McAllen', True, 'McAllen', 'TX', 78501, 'USA', 26.218121, -98.237994), 98: ('Charleston', True, 'Charleston', 'SC', 29401, 'USA', 32.778908, -79.937993), 99: ('Columbia', True, 'Columbia', 'SC', 29201, 'USA', 33.984705, -81.028435), 100: ('Vallejo', True, 'Vallejo', 'CA', 94589, 'USA', 38.177394, -122.239773)}

    #{'Warehouse ID': ('Name', 'Active', 'Status', 'City', 'State', 'Zip Code', 'Country or Region', 'Latitude', 'Longitude')}
    warehouses = {1: ('Los Angeles', False, 'Potential', 'Los Angeles', 'CA', 90001, 'USA', 33.974044, -118.248849), 2: ('New York City', False, 'Potential', 'New York City', 'NY', 10001, 'USA', 40.75012, -73.997846), 3: ('Chicago', False, 'Potential', 'Chicago', 'IL', 60602, 'USA', 41.88331, -87.624713), 7: ('Houston', False, 'Potential', 'Houston', 'TX', 77001, 'USA', 29.757317, -95.365005), 8: ('Atlanta', False, 'Potential', 'Atlanta', 'GA', 30301, 'USA', 33.753693, -84.389544), 9: ('Dallas', False, 'Potential', 'Dallas', 'TX', 75201, 'USA', 32.787642, -96.799525), 11: ('Phoenix', True, 'Fixed', 'Phoenix', 'AZ', 85001, 'USA', 33.451015, -112.068554), 14: ('New York', True, 'Fixed', 'Nassau-Suffolk', 'NY', 12123, 'USA', 42.526891, -73.609283), 15: ('St. Louis', False, 'Potential', 'St. Louis', 'MO', 63101, 'USA', 38.631358, -90.192246), 16: ('Baltimore', False, 'Potential', 'Baltimore', 'MD', 21201, 'USA', 39.294398, -76.622747), 17: ('Pittsburgh', False, 'Potential', 'Pittsburgh', 'PA', 15201, 'USA', 40.474802, -79.95449), 19: ('Seattle', False, 'Potential', 'Seattle', 'WA', 98101, 'USA', 47.611601, -122.333038), 25: ('Denver', False, 'Potential', 'Denver', 'CO', 80201, 'USA', 39.75071, -104.996225), 27: ('Kansas City', False, 'Potential', 'Kansas City', 'MO', 64101, 'USA', 39.103883, -94.600613), 28: ('San Francisco', False, 'Potential', 'San Francisco', 'CA', 94102, 'USA', 37.779887, -122.418066), 38: ('Orlando', False, 'Potential', 'Orlando', 'FL', 32801, 'USA', 28.541627, -81.37397), 39: ('Columbus', False, 'Potential', 'Columbus', 'OH', 43085, 'USA', 40.099395, -83.016823), 43: ('New Orleans', False, 'Potential', 'New Orleans', 'LA', 70112, 'USA', 29.956664, -90.077506), 44: ('Las Vegas', False, 'Potential', 'Las Vegas', 'NV', 89101, 'USA', 36.17269, -115.121117), 45: ('Salt Lake City', False, 'Potential', 'Salt Lake City', 'UT', 84101, 'USA', 40.755074, -111.89849), 52: ('Memphis', False, 'Potential', 'Memphis', 'TN', 37501, 'USA', 35.033731, -89.934319), 55: ('Raleigh', False, 'Potential', 'Raleigh', 'NC', 27601, 'USA', 35.773856, -78.634051), 57: ('Oklahoma City', False, 'Potential', 'Oklahoma City', 'OK', 73101, 'USA', 35.467763, -97.520977), 60: ('Louisville', False, 'Potential', 'Louisville', 'KY', 40201, 'USA', 38.253601, -85.768124), 99: ('Columbia', False, 'Potential', 'Columbia', 'SC', 29201, 'USA', 33.984705, -81.028435)}

    #{'Plant ID': ('Name', 'City', 'State', 'Zip Code', 'Country or Region', 'Latitude', 'Longitude')}
    plants = {1: ('Plant 1', 'Ashland', 'KY', 'nan', 'USA', 38.448338, -82.66621175), 2: ('Plant 2', 'Dayton', 'OH', 'nan', 'USA', 39.74513948979592, -84.18873704081632), 3: ('Plant 3', 'Baltimore ', 'MD', 'nan', 'USA', 39.31362598484849, -76.61557987878788), 4: ('Plant 4- Imports', 'Long Beach', 'CA', 'nan', 'USA', 33.79856248571428, -118.18454957142855)}
    
    # {'Product ID': ('Name')}
    products = {1: 'Product Family 1', 2: 'Product Family 2',  3: 'Product Family 3', 4: 'Product Family 4'}
    
    #{('Customer ID', 'Product ID'): 'Demand in Tons'}
    customer_demands = {(1, 1): 32007.5, (1, 2): 22862.5, (1, 3): 18290.0, (1, 4): 13717.5, (2, 1): 30138.5, (2, 2): 21527.5, (2, 3): 17222.0, (2, 4): 12916.5, (3, 1): 27205.5, (3, 2): 19432.5, (3, 3): 15546.0, (3, 4): 11659.5, (4, 1): 20394.5, (4, 2): 14567.5, (4, 3): 11654.0, (4, 4): 8740.5, (5, 1): 16110.5, (5, 2): 11507.5, (5, 3): 9206.0, (5, 4): 6904.5, (6, 1): 15620.5, (6, 2): 11157.5, (6, 3): 8926.0, (6, 4): 6694.5, (7, 1): 13478.5, (7, 2): 9627.5, (7, 3): 7702.0, (7, 4): 5776.5, (8, 1): 12694.5, (8, 2): 9067.5, (8, 3): 7254.0, (8, 4): 5440.5, (9, 1): 10941.0, (9, 2): 7815.0, (9, 3): 6252.0, (9, 4): 4689.0, (10, 1): 10720.5, (10, 2): 7657.5, (10, 3): 6126.0, (10, 4): 4594.5, (11, 1): 9936.5, (11, 2): 7097.5, (11, 3): 5678.0, (11, 4): 4258.5, (12, 1): 9772.0, (12, 2): 6980.0, (12, 3): 5584.0, (12, 4): 4188.0, (13, 1): 9359.0, (13, 2): 6685.0, (13, 3): 5348.0, (13, 4): 4011.0, (14, 1): 9331.0, (14, 2): 6665.0, (14, 3): 5332.0, (14, 4): 3999.0, (15, 1): 8949.5, (15, 2): 6392.5, (15, 3): 5114.0, (15, 4): 3835.5, (16, 1): 8662.5, (16, 2): 6187.5, (16, 3): 4950.0, (16, 4): 3712.5, (17, 1): 8263.5, (17, 2): 5902.5, (17, 3): 4722.0, (17, 4): 3541.5, (18, 1): 7945.0, (18, 2): 5675.0, (18, 3): 4540.0, (18, 4): 3405.0, (19, 1): 7938.0, (19, 2): 5670.0, (19, 3): 4536.0, (19, 4): 3402.0, (20, 1): 7794.5, (20, 2): 5567.5, (20, 3): 4454.0, (20, 4): 3340.5, (21, 1): 7794.5, (21, 2): 5567.5, (21, 3): 4454.0, (21, 4): 3340.5, (22, 1): 7787.5, (22, 2): 5562.5, (22, 3): 4450.0, (22, 4): 3337.5, (23, 1): 7154.0, (23, 2): 5110.0, (23, 3): 4088.0, (23, 4): 3066.0, (24, 1): 6800.5, (24, 2): 4857.5, (24, 3): 3886.0, (24, 4): 2914.5, (25, 1): 6653.5, (25, 2): 4752.5, (25, 3): 3802.0, (25, 4): 2851.5, (26, 1): 6254.5, (26, 2): 4467.5, (26, 3): 3574.0, (26, 4): 2680.5, (27, 1): 5981.5, (27, 2): 4272.5, (27, 3): 3418.0, (27, 4): 2563.5, (28, 1): 5817.0, (28, 2): 4155.0, (28, 3): 3324.0, (28, 4): 2493.0, (29, 1): 5687.5, (29, 2): 4062.5, (29, 3): 3250.0, (29, 4): 2437.5, (30, 1): 5631.5, (30, 2): 4022.5, (30, 3): 3218.0, (30, 4): 2413.5, (31, 1): 5624.5, (31, 2): 4017.5, (31, 3): 3214.0, (31, 4): 2410.5, (32, 1): 5446.0, (32, 2): 3890.0, (32, 3): 3112.0, (32, 4): 2334.0, (33, 1): 5404.0, (33, 2): 3860.0, (33, 3): 3088.0, (33, 4): 2316.0, (34, 1): 5288.5, (34, 2): 3777.5, (34, 3): 3022.0, (34, 4): 2266.5, (35, 1): 5260.5, (35, 2): 3757.5, (35, 3): 3006.0, (35, 4): 2254.5, (36, 1): 5260.5, (36, 2): 3757.5, (36, 3): 3006.0, (36, 4): 2254.5, (37, 1): 5145.0, (37, 2): 3675.0, (37, 3): 2940.0, (37, 4): 2205.0, (38, 1): 5134.5, (38, 2): 3667.5, (38, 3): 2934.0, (38, 4): 2200.5, (39, 1): 5110.0, (39, 2): 3650.0, (39, 3): 2920.0, (39, 4): 2190.0, (40, 1): 5078.5, (40, 2): 3627.5, (40, 3): 2902.0, (40, 4): 2176.5, (41, 1): 4725.0, (41, 2): 3375.0, (41, 3): 2700.0, (41, 4): 2025.0, (42, 1): 4672.5, (42, 2): 3337.5, (42, 3): 2670.0, (42, 4): 2002.5, (43, 1): 4574.5, (43, 2): 3267.5, (43, 3): 2614.0, (43, 4): 1960.5, (44, 1): 4417.0, (44, 2): 3155.0, (44, 3): 2524.0, (44, 4): 1893.0, (45, 1): 4364.5, (45, 2): 3117.5, (45, 3): 2494.0, (45, 4): 1870.5, (46, 1): 4074.0, (46, 2): 2910.0, (46, 3): 2328.0, (46, 4): 1746.0, (47, 1): 4032.0, (47, 2): 2880.0, (47, 3): 2304.0, (47, 4): 1728.0, (48, 1): 3969.0, (48, 2): 2835.0, (48, 3): 2268.0, (48, 4): 1701.0, (49, 1): 3867.5, (49, 2): 2762.5, (49, 3): 2210.0, (49, 4): 1657.5, (50, 1): 3867.5, (50, 2): 2762.5, (50, 3): 2210.0, (50, 4): 1657.5, (51, 1): 3801.0, (51, 2): 2715.0, (51, 3): 2172.0, (51, 4): 1629.0, (52, 1): 3790.5, (52, 2): 2707.5, (52, 3): 2166.0, (52, 4): 1624.5, (53, 1): 3766.0, (53, 2): 2690.0, (53, 3): 2152.0, (53, 4): 1614.0, (54, 1): 3748.5, (54, 2): 2677.5, (54, 3): 2142.0, (54, 4): 1606.5, (55, 1): 3675.0, (55, 2): 2625.0, (55, 3): 2100.0, (55, 4): 1575.0, (56, 1): 3619.0, (56, 2): 2585.0, (56, 3): 2068.0, (56, 4): 1551.0, (57, 1): 3605.0, (57, 2): 2575.0, (57, 3): 2060.0, (57, 4): 1545.0, (58, 1): 3591.0, (58, 2): 2565.0, (58, 3): 2052.0, (58, 4): 1539.0, (59, 1): 3563.0, (59, 2): 2545.0, (59, 3): 2036.0, (59, 4): 1527.0, (60, 1): 3475.5, (60, 2): 2482.5, (60, 3): 1986.0, (60, 4): 1489.5, (61, 1): 3304.0, (61, 2): 2360.0, (61, 3): 1888.0, (61, 4): 1416.0, (62, 1): 3300.5, (62, 2): 2357.5, (62, 3): 1886.0, (62, 4): 1414.5, (63, 1): 3290.0, (63, 2): 2350.0, (63, 3): 1880.0, (63, 4): 1410.0, (64, 1): 3164.0, (64, 2): 2260.0, (64, 3): 1808.0, (64, 4): 1356.0, (65, 1): 3164.0, (65, 2): 2260.0, (65, 3): 1808.0, (65, 4): 1356.0, (66, 1): 3150.0, (66, 2): 2250.0, (66, 3): 1800.0, (66, 4): 1350.0, (67, 1): 3066.0, (67, 2): 2190.0, (67, 3): 1752.0, (67, 4): 1314.0, (68, 1): 3038.0, (68, 2): 2170.0, (68, 3): 1736.0, (68, 4): 1302.0, (69, 1): 2978.5, (69, 2): 2127.5, (69, 3): 1702.0, (69, 4): 1276.5, (70, 1): 2730.0, (70, 2): 1950.0, (70, 3): 1560.0, (70, 4): 1170.0, (71, 1): 2674.0, (71, 2): 1910.0, (71, 3): 1528.0, (71, 4): 1146.0, (72, 1): 2590.0, (72, 2): 1850.0, (72, 3): 1480.0, (72, 4): 1110.0, (73, 1): 2537.5, (73, 2): 1812.5, (73, 3): 1450.0, (73, 4): 1087.5, (74, 1): 2453.5, (74, 2): 1752.5, (74, 3): 1402.0, (74, 4): 1051.5, (75, 1): 2404.5, (75, 2): 1717.5, (75, 3): 1374.0, (75, 4): 1030.5, (76, 1): 2387.0, (76, 2): 1705.0, (76, 3): 1364.0, (76, 4): 1023.0, (77, 1): 2359.0, (77, 2): 1685.0, (77, 3): 1348.0, (77, 4): 1011.0, (78, 1): 2324.0, (78, 2): 1660.0, (78, 3): 1328.0, (78, 4): 996.0, (79, 1): 2289.0, (79, 2): 1635.0, (79, 3): 1308.0, (79, 4): 981.0, (80, 1): 2198.0, (80, 2): 1570.0, (80, 3): 1256.0, (80, 4): 942.0, (81, 1): 2180.5, (81, 2): 1557.5, (81, 3): 1246.0, (81, 4): 934.5, (82, 1): 2173.5, (82, 2): 1552.5, (82, 3): 1242.0, (82, 4): 931.5, (83, 1): 2152.5, (83, 2): 1537.5, (83, 3): 1230.0, (83, 4): 922.5, (84, 1): 2145.5, (84, 2): 1532.5, (84, 3): 1226.0, (84, 4): 919.5, (85, 1): 2138.5, (85, 2): 1527.5, (85, 3): 1222.0, (85, 4): 916.5, (86, 1): 2082.5, (86, 2): 1487.5, (86, 3): 1190.0, (86, 4): 892.5, (87, 1): 2068.5, (87, 2): 1477.5, (87, 3): 1182.0, (87, 4): 886.5, (88, 1): 1995.0, (88, 2): 1425.0, (88, 3): 1140.0, (88, 4): 855.0, (89, 1): 1942.5, (89, 2): 1387.5, (89, 3): 1110.0, (89, 4): 832.5, (90, 1): 1932.0, (90, 2): 1380.0, (90, 3): 1104.0, (90, 4): 828.0, (91, 1): 1928.5, (91, 2): 1377.5, (91, 3): 1102.0, (91, 4): 826.5, (92, 1): 1897.0, (92, 2): 1355.0, (92, 3): 1084.0, (92, 4): 813.0, (93, 1): 1886.5, (93, 2): 1347.5, (93, 3): 1078.0, (93, 4): 808.5, (94, 1): 1883.0, (94, 2): 1345.0, (94, 3): 1076.0, (94, 4): 807.0, (95, 1): 1855.0, (95, 2): 1325.0, (95, 3): 1060.0, (95, 4): 795.0, (96, 1): 1844.5, (96, 2): 1317.5, (96, 3): 1054.0, (96, 4): 790.5, (97, 1): 1785.0, (97, 2): 1275.0, (97, 3): 1020.0, (97, 4): 765.0, (98, 1): 1781.5, (98, 2): 1272.5, (98, 3): 1018.0, (98, 4): 763.5, (99, 1): 1760.5, (99, 2): 1257.5, (99, 3): 1006.0, (99, 4): 754.5, (100, 1): 1715.0, (100, 2): 1225.0, (100, 3): 980.0, (100, 4): 735.0}

    #{('Plant ID', 'Product ID'): Capacity, dtype='object')}
    plant_product_info = {(1, 1): 210000000, 
                          (2, 2): 210000000, 
                          (3, 3): 210000000, 
                          (4, 4): 210000000}

    #{('Plant ID', 'Warehouse'): 'Distance'}
    plant_wh_distance = {(1, 8): 398.9338, (1, 16): 389.416, (1, 3): 416.5495, (1, 99): 379.2462, (1, 39): 136.3223, (1, 9): 1041.9197, (1, 25): 1412.1889, (1, 7): 1109.511, (1, 27): 759.3619, (1, 44): 2100.3951, (1, 1): 2352.4923, (1, 60): 198.8163, (1, 52): 549.8012, (1, 43): 852.5521, (1, 14): 651.6674, (1, 2): 575.2765, (1, 57): 995.8787, (1, 38): 811.8669, (1, 11): 1972.6675, (1, 17): 237.3652, (1, 55): 340.6886, (1, 45): 1836.1098, (1, 28): 2528.696, (1, 19): 2447.402, (1, 15): 479.7045, (2, 8): 488.3189, (2, 16): 476.7873, (2, 3): 274.249, (2, 99): 512.4434, (2, 39): 78.7071, (2, 9): 1002.1928, (2, 25): 1300.4738, (2, 7): 1103.5674, (2, 27): 656.9782, (2, 44): 1998.1415, (2, 1): 2255.292, (2, 60): 157.3716, (2, 52): 534.244, (2, 43): 888.8501, (2, 14): 687.0221, (2, 2): 638.6194, (2, 57): 927.2277, (2, 38): 932.1641, (2, 11): 1885.7423, (2, 17): 270.4273, (2, 55): 482.2012, (2, 45): 1717.8791, (2, 28): 2415.5119, (2, 19): 2311.3811, (2, 15): 389.7331, (3, 8): 680.8196, (3, 16): 1.6304, (3, 3): 711.8986, (3, 99): 521.0856, (3, 39): 406.2093, (3, 9): 1427.9477, (3, 25): 1776.2335, (3, 7): 1475.2079, (3, 27): 1133.6396, (3, 44): 2475.1895, (3, 1): 2731.7993, (3, 60): 587.3935, (3, 52): 931.129, (3, 43): 1178.9456, (3, 14): 320.5585, (3, 2): 200.895, (3, 57): 1385.6421, (3, 38): 934.3305, (3, 11): 2358.2677, (3, 17): 229.1305, (3, 55): 316.4794, (3, 45): 2189.2615, (3, 28): 2889.7044, (3, 19): 2744.2981, (3, 15): 860.9467, (4, 8): 2278.1763, (4, 16): 2732.6718, (4, 3): 2060.8105, (4, 99): 2498.8565, (4, 39): 2330.6249, (4, 9): 1456.0677, (4, 25): 986.3866, (4, 7): 1610.8561, (4, 27): 1599.7023, (4, 44): 281.4448, (4, 1): 14.9439, (4, 60): 2155.2973, (4, 52): 1895.0926, (4, 43): 1963.387, (4, 14): 2907.302, (4, 2): 2888.9208, (4, 57): 1389.3295, (4, 38): 2587.7258, (4, 11): 415.8209, (4, 17): 2521.3253, (4, 55): 2633.4084, (4, 45): 697.6278, (4, 28): 428.2596, (4, 19): 1153.7434, (4, 15): 1873.9433}

    #{('Warehouse ID', 'Customer ID'): 'Distance'}
    wh_cust_distance = {(8, 76): 622.1157, (8, 67): 993.2936, (8, 77): 1496.685, (8, 84): 803.2868, (8, 93): 694.2955, (8, 8): 0.0, (8, 54): 964.2637, (8, 80): 2315.2362, (8, 16): 679.4783, (8, 88): 540.7419, (8, 42): 880.4291, (8, 66): 165.2179, (8, 4): 1103.8727, (8, 46): 823.3119, (8, 98): 313.4469, (8, 41): 266.5292, (8, 3): 694.1537, (8, 31): 436.1815, (8, 22): 654.2633, (8, 99): 228.1406, (8, 39): 524.6581, (8, 9): 848.4912, (8, 61): 489.3774, (8, 25): 1427.1617, (8, 6): 705.2432, (8, 74): 1519.2902, (8, 69): 310.3789, (8, 37): 690.6234, (8, 32): 884.6281, (8, 68): 2354.2865, (8, 81): 665.3979, (8, 58): 754.8611, (8, 47): 361.9032, (8, 64): 160.6501, (8, 83): 720.578, (8, 49): 997.6178, (8, 7): 826.5263, (8, 35): 503.1607, (8, 56): 329.445, (8, 91): 878.8444, (8, 27): 797.9865, (8, 79): 180.3248, (8, 44): 2052.7972, (8, 90): 537.2929, (8, 1): 2280.1462, (8, 60): 377.6997, (8, 97): 1152.4073, (8, 52): 387.0249, (8, 23): 714.0043, (8, 50): 848.0977, (8, 40): 788.9952, (8, 12): 1070.9874, (8, 96): 354.3752, (8, 53): 862.3246, (8, 48): 252.8114, (8, 14): 992.4217, (8, 29): 961.5574, (8, 43): 500.4799, (8, 2): 881.5245, (8, 24): 872.1965, (8, 33): 594.3405, (8, 20): 2505.0039, (8, 57): 890.8391, (8, 75): 963.336, (8, 13): 2261.9847, (8, 38): 473.7823, (8, 63): 783.2551, (8, 11): 1872.7478, (8, 17): 618.554, (8, 26): 2557.6556, (8, 65): 1061.0586, (8, 55): 418.8167, (8, 62): 553.8429, (8, 10): 2221.412, (8, 51): 880.0651, (8, 36): 2454.8561, (8, 45): 1863.9411, (8, 34): 1040.1332, (8, 18): 2225.5849, (8, 28): 2517.8998, (8, 30): 2486.8359, (8, 94): 539.0454, (8, 82): 834.9588, (8, 19): 2568.4667, (8, 87): 1016.6803, (8, 15): 550.6599, (8, 92): 2454.0457, (8, 72): 920.6697, (8, 78): 2568.0399, (8, 21): 491.3911, (8, 85): 645.2058, (8, 70): 1815.886, (8, 71): 797.3859, (8, 100): 2504.298, (8, 73): 2347.3058, (8, 5): 640.3516, (8, 59): 675.4229, (8, 95): 913.2894, (8, 89): 754.943, (8, 86): 645.4024, (16, 76): 336.7306, (16, 67): 325.7224, (16, 77): 1969.3428, (16, 84): 128.6197, (16, 93): 506.4952, (16, 8): 679.4783, (16, 54): 1587.5123, (16, 80): 2739.5764, (16, 16): 0.0, (16, 88): 1208.6516, (16, 42): 200.9696, (16, 66): 816.1792, (16, 4): 424.4712, (16, 46): 324.6231, (16, 98): 573.8975, (16, 41): 431.0668, (16, 3): 712.0294, (16, 31): 497.4067, (16, 22): 360.7192, (16, 99): 519.5429, (16, 39): 406.0708, (16, 9): 1427.092, (16, 61): 476.773, (16, 25): 1776.0686, (16, 6): 468.1432, (16, 74): 2061.8341, (16, 69): 464.8263, (16, 37): 1099.5834, (16, 32): 1460.3332, (16, 68): 2751.8318, (16, 81): 688.5126, (16, 58): 629.3322, (16, 47): 331.7556, (16, 64): 521.76, (16, 83): 80.6736, (16, 49): 318.2386, (16, 7): 1474.1129, (16, 35): 597.0953, (16, 56): 808.0277, (16, 91): 199.4486, (16, 27): 1133.3231, (16, 79): 549.6889, (16, 44): 2474.9157, (16, 90): 1081.9513, (16, 1): 2731.4614, (16, 60): 586.7938, (16, 97): 1816.4003, (16, 52): 930.22, (16, 23): 1127.9764, (16, 50): 168.6221, (16, 40): 756.0188, (16, 12): 1104.7166, (16, 96): 1033.853, (16, 53): 185.6583, (16, 48): 701.61, (16, 14): 322.1078, (16, 29): 282.2726, (16, 43): 1177.6565, (16, 2): 202.1919, (16, 24): 192.7373, (16, 33): 200.1055, (16, 20): 2877.2835, (16, 57): 1385.0165, (16, 75): 1208.3764, (16, 13): 2716.614, (16, 38): 932.7073, (16, 63): 104.5419, (16, 11): 2357.8029, (16, 17): 229.4013, (16, 26): 2778.4509, (16, 65): 382.5341, (16, 55): 314.8685, (16, 62): 152.1699, (16, 10): 2675.7157, (16, 51): 321.2725, (16, 36): 2814.1783, (16, 45): 2189.2173, (16, 34): 1669.9897, (16, 18): 2701.7916, (16, 28): 2889.6007, (16, 30): 2866.9573, (16, 94): 1052.8764, (16, 82): 177.5807, (16, 19): 2744.677, (16, 87): 337.9741, (16, 15): 860.5126, (16, 92): 2825.5776, (16, 72): 306.8063, (16, 78): 2751.933, (16, 21): 1004.505, (16, 85): 469.8507, (16, 70): 2327.9521, (16, 71): 1270.6236, (16, 100): 2869.3073, (16, 73): 2788.3456, (16, 5): 40.0657, (16, 59): 1081.0167, (16, 95): 1324.3849, (16, 89): 76.8491, (16, 86): 290.5168, (3, 76): 378.5994, (3, 67): 837.996, (3, 77): 1329.8157, (3, 84): 751.2087, (3, 93): 230.9843, (3, 8): 694.1537, (3, 54): 1155.2809, (3, 80): 2054.4004, (3, 16): 712.0294, (3, 88): 960.3392, (3, 42): 828.531, (3, 66): 683.5145, (3, 4): 1000.5751, (3, 46): 532.1092, (3, 98): 892.5353, (3, 41): 695.6897, (3, 3): 0.0, (3, 31): 297.7207, (3, 22): 363.5476, (3, 99): 770.0103, (3, 39): 318.4073, (3, 9): 948.7264, (3, 61): 273.6028, (3, 25): 1083.1767, (3, 6): 278.4286, (3, 74): 1475.4107, (3, 69): 800.7509, (3, 37): 1378.9934, (3, 32): 972.2752, (3, 68): 2057.2611, (3, 81): 30.5187, (3, 58): 147.7036, (3, 47): 686.4921, (3, 64): 662.8425, (3, 83): 672.4704, (3, 49): 906.894, (3, 7): 1111.2466, (3, 35): 195.5796, (3, 56): 1014.163, (3, 91): 834.6314, (3, 27): 487.7269, (3, 79): 534.7456, (3, 44): 1792.8548, (3, 90): 652.8291, (3, 1): 2058.0768, (3, 60): 317.5506, (3, 97): 1460.8708, (3, 52): 577.1177, (3, 23): 1403.6693, (3, 50): 810.3692, (3, 40): 95.6624, (3, 12): 418.6941, (3, 96): 912.1633, (3, 53): 845.9949, (3, 48): 469.0608, (3, 14): 846.5004, (3, 29): 895.8001, (3, 43): 984.9639, (3, 2): 838.0565, (3, 24): 827.6543, (3, 33): 822.0953, (3, 20): 2175.0256, (3, 57): 817.2135, (3, 75): 508.7052, (3, 13): 2045.0849, (3, 38): 1163.0398, (3, 63): 780.0156, (3, 11): 1711.8393, (3, 17): 483.9595, (3, 26): 2070.0156, (3, 65): 982.6338, (3, 55): 756.4496, (3, 62): 729.3872, (3, 10): 2004.7238, (3, 51): 610.4127, (3, 36): 2109.6439, (3, 45): 1483.1968, (3, 34): 1240.1333, (3, 18): 2042.0595, (3, 28): 2187.1327, (3, 30): 2166.5993, (3, 94): 1233.0383, (3, 82): 726.8092, (3, 19): 2043.5697, (3, 87): 908.9044, (3, 15): 309.2874, (3, 92): 2123.5115, (3, 72): 695.5059, (3, 78): 2049.2961, (3, 21): 1185.5448, (3, 85): 248.4582, (3, 70): 1702.4509, (3, 71): 704.8479, (3, 100): 2165.5573, (3, 73): 2109.3901, (3, 5): 700.7041, (3, 59): 1362.8319, (3, 95): 696.882, (3, 89): 765.1081, (3, 86): 430.3261, (99, 76): 576.0087, (99, 67): 844.9789, (99, 77): 1717.0031, (99, 84): 648.1345, (99, 93): 698.1113, (99, 8): 228.1406, (99, 54): 1190.6882, (99, 80): 2532.0977, (99, 16): 519.5429, (99, 88): 756.7229, (99, 42): 715.6219, (99, 66): 393.357, (99, 4): 933.515, (99, 46): 738.7527, (99, 98): 123.1019, (99, 41): 99.4819, (99, 3): 770.0103, (99, 31): 475.0454, (99, 22): 612.9921, (99, 99): 0.0, (99, 39): 514.65, (99, 9): 1076.2389, (99, 61): 513.3828, (99, 25): 1625.6663, (99, 6): 693.522, (99, 74): 1746.4619, (99, 69): 82.9309, (99, 37): 643.8076, (99, 32): 1112.3273, (99, 68): 2567.3406, (99, 81): 739.5141, (99, 58): 788.6434, (99, 47): 189.6164, (99, 64): 116.1455, (99, 83): 578.073, (99, 49): 831.8411, (99, 7): 1048.9005, (99, 35): 576.0715, (99, 56): 302.0233, (99, 91): 712.1627, (99, 27): 980.0284, (99, 79): 255.9384, (99, 44): 2268.4667, (99, 90): 758.7657, (99, 1): 2500.446, (99, 60): 467.0477, (99, 97): 1365.468, (99, 52): 603.7796, (99, 23): 671.1651, (99, 50): 683.1233, (99, 40): 857.6553, (99, 12): 1178.7381, (99, 96): 552.1696, (99, 53): 690.0766, (99, 48): 422.1802, (99, 14): 841.6309, (99, 29): 792.4903, (99, 43): 705.8937, (99, 2): 714.3601, (99, 24): 706.3229, (99, 33): 391.6411, (99, 20): 2714.5556, (99, 57): 1109.646, (99, 75): 1127.5063, (99, 13): 2482.6422, (99, 38): 444.1173, (99, 63): 616.9858, (99, 11): 2095.8793, (99, 17): 533.3264, (99, 26): 2737.9762, (99, 65): 887.5183, (99, 55): 216.4556, (99, 62): 374.4202, (99, 10): 2441.9302, (99, 51): 779.1926, (99, 36): 2662.0563, (99, 45): 2061.702, (99, 34): 1265.3594, (99, 18): 2448.5904, (99, 28): 2727.4008, (99, 30): 2697.5977, (99, 94): 551.4882, (99, 82): 691.635, (99, 19): 2738.7144, (99, 87): 853.7027, (99, 15): 710.2905, (99, 92): 2663.2953, (99, 72): 800.4865, (99, 78): 2740.0723, (99, 21): 501.3168, (99, 85): 644.7555, (99, 70): 2041.445, (99, 71): 1011.1585, (99, 100): 2712.6425, (99, 73): 2566.4869, (99, 5): 479.5685, (99, 59): 625.9091, (99, 95): 1116.8106, (99, 89): 588.9034, (99, 86): 580.2031, (39, 76): 120.2079, (39, 67): 603.2065, (39, 77): 1574.0452, (39, 84): 470.2017, (39, 93): 183.4672, (39, 8): 524.6581, (39, 54): 1262.9554, (39, 80): 2334.8828, (39, 16): 406.0708, (39, 88): 954.7762, (39, 42): 554.034, (39, 66): 590.3059, (39, 4): 755.1265, (39, 46): 339.7797, (39, 98): 629.5205, (39, 41): 423.8346, (39, 3): 318.4073, (39, 31): 123.3155, (39, 22): 140.5253, (39, 99): 514.65, (39, 39): 0.0, (39, 9): 1080.0735, (39, 61): 78.2701, (39, 25): 1370.0799, (39, 6): 183.1044, (39, 74): 1683.724, (39, 69): 523.4237, (39, 37): 1155.6096, (39, 32): 1110.4192, (39, 68): 2345.9685, (39, 81): 291.4814, (39, 58): 283.5545, (39, 47): 388.7983, (39, 64): 429.6875, (39, 83): 382.114, (39, 49): 651.2511, (39, 7): 1176.8158, (39, 35): 194.9852, (39, 56): 798.9818, (39, 91): 558.1809, (39, 27): 731.1507, (39, 79): 344.4531, (39, 44): 2070.5863, (39, 90): 740.4582, (39, 1): 2329.069, (39, 60): 229.783, (39, 97): 1530.6505, (39, 52): 607.8242, (39, 23): 1182.415, (39, 50): 530.0526, (39, 40): 382.6295, (39, 12): 732.1461, (39, 96): 835.5666, (39, 53): 562.1437, (39, 48): 400.9025, (39, 14): 608.3235, (39, 29): 630.6831, (39, 43): 950.3922, (39, 2): 561.6326, (39, 24): 550.7741, (39, 33): 503.7116, (39, 20): 2471.2895, (39, 57): 1005.7562, (39, 75): 802.8793, (39, 13): 2314.6278, (39, 38): 948.0641, (39, 63): 487.6432, (39, 11): 1961.8962, (39, 17): 192.7526, (39, 26): 2387.1623, (39, 65): 727.1718, (39, 55): 450.9867, (39, 62): 410.9801, (39, 10): 2273.8124, (39, 51): 413.6298, (39, 36): 2408.4255, (39, 45): 1783.9265, (39, 34): 1348.1916, (39, 18): 2302.7245, (39, 28): 2483.6196, (39, 30): 2460.8915, (39, 94): 1040.3301, (39, 82): 462.9359, (39, 19): 2361.9656, (39, 87): 659.7342, (39, 15): 467.3757, (39, 92): 2419.576, (39, 72): 482.1309, (39, 78): 2367.6248, (39, 21): 990.1967, (39, 85): 130.1146, (39, 70): 1938.1098, (39, 71): 889.6713, (39, 100): 2463.4446, (39, 73): 2384.8163, (39, 5): 389.2664, (39, 59): 1138.0522, (39, 95): 927.8804, (39, 89): 468.2414, (39, 86): 167.5956, (9, 76): 1198.2405, (9, 67): 1681.1678, (9, 77): 691.0202, (9, 84): 1526.6985, (9, 93): 1136.5918, (9, 8): 848.4912, 
                        (9, 54): 215.2703, (9, 80): 1510.9044, (9, 16): 1427.092, (9, 88): 433.3719, (9, 42): 1611.9602, (9, 66): 683.7971, (9, 4): 1826.7128, (9, 46): 1411.7608, (9, 98): 1153.73, (9, 41): 1095.418, (9, 3): 948.7264, (9, 31): 959.4802, (9, 22): 1208.7722, (9, 99): 1076.2389, (9, 39): 1080.0735, (9, 9): 0.0, (9, 61): 1002.7753, (9, 25): 781.4251, (9, 6): 1177.8578, (9, 74): 671.7984, (9, 69): 1157.583, (9, 37): 1299.3773, (9, 32): 36.2731, (9, 68): 1567.8831, (9, 81): 943.49, (9, 58): 1094.0486, (9, 47): 1173.7942, (9, 64): 987.8809, (9, 83): 1435.0206, (9, 49): 1719.7984, (9, 7): 266.3247, (9, 35): 902.6999, (9, 56): 1057.1105, (9, 91): 1613.9114, (9, 27): 534.6014, (9, 79): 896.1645, (9, 44): 1258.9387, (9, 90): 345.2519, (9, 1): 1459.7986, (9, 60): 855.477, (9, 97): 544.8527, (9, 52): 498.8168, (9, 23): 1308.5027, (9, 50): 1583.2976, (9, 40): 1010.3116, (9, 12): 1018.4377, (9, 96): 630.0386, (9, 53): 1608.8777, (9, 48): 726.8706, (9, 14): 1685.3675, (9, 29): 1693.3006, (9, 43): 521.2348, (9, 2): 1617.1933, (9, 24): 1606.4849, (9, 33): 1408.0059, (9, 20): 1732.7989, (9, 57): 223.7116, (9, 75): 692.8218, (9, 13): 1440.1216, (9, 38): 1133.7379, (9, 63): 1527.7348, (9, 11): 1042.373, (9, 17): 1263.8314, (9, 26): 1923.0516, (9, 65): 1793.6112, (9, 55): 1244.9447, (9, 62): 1343.6864, (9, 10): 1400.5738, (9, 51): 1489.2946, (9, 36): 1694.9633, (9, 45): 1177.3575, (9, 34): 295.997, (9, 18): 1393.2248, (9, 28): 1745.7004, (9, 30): 1709.1835, (9, 94): 1099.5531, (9, 82): 1533.3095, (9, 19): 1980.2485, (9, 87): 1731.5991, (9, 15): 645.9453, (9, 92): 1684.2617, (9, 72): 1561.7673, (9, 78): 1971.5917, (9, 21): 1081.1264, (9, 85): 1120.143, (9, 70): 974.1226, (9, 71): 279.4146, (9, 100): 1737.8708, (9, 73): 1531.567, (9, 5): 1394.764, (9, 59): 1293.5184, (9, 95): 400.6993, (9, 89): 1502.2907, (9, 86): 1247.6577, (25, 76): 1455.7808, (25, 67): 1918.9627, (25, 77): 394.9978, (25, 84): 1830.6692, (25, 93): 1313.2645, (25, 8): 1427.1617, (25, 54): 910.7517, (25, 80): 971.6092, (25, 16): 1776.0686, (25, 88): 1189.7567, (25, 42): 1910.089, (25, 66): 1289.8569, (25, 4): 2082.229, (25, 46): 1611.4108, (25, 98): 1734.4404, (25, 41): 1600.9985, (25, 3): 1083.1767, (25, 31): 1288.1258, (25, 22): 1444.9668, (25, 99): 1625.6663, (25, 39): 1370.0799, (25, 9): 781.4251, (25, 61): 1300.4026, (25, 25): 0.0, (25, 6): 1360.4355, (25, 74): 658.2951, (25, 69): 1693.2613, (25, 37): 2019.9441, (25, 32): 759.8848, (25, 68): 976.0984, (25, 81): 1100.6015, (25, 58): 1208.5517, (25, 47): 1643.5791, (25, 64): 1513.1613, (25, 83): 1747.6759, (25, 49): 1989.9814, (25, 7): 1037.1401, (25, 35): 1180.6625, (25, 56): 1717.9088, (25, 91): 1915.7989, (25, 27): 655.9696, (25, 79): 1380.9796, (25, 44): 712.1544, (25, 90): 918.8031, (25, 1): 982.2847, (25, 60): 1221.1498, (25, 97): 1194.4046, (25, 52): 1046.2981, (25, 23): 2033.894, (25, 50): 1890.4029, (25, 40): 1075.6149, (25, 12): 823.4909, (25, 96): 1344.6542, (25, 53): 1925.0723, (25, 48): 1204.1271, (25, 14): 1927.9794, (25, 29): 1978.8, (25, 43): 1274.2881, (25, 2): 1919.2462, (25, 24): 1908.6879, (25, 33): 1842.388, (25, 20): 1103.9817, (25, 57): 595.0775, (25, 75): 574.5096, (25, 13): 971.139, (25, 38): 1827.6375, (25, 63): 1854.909, (25, 11): 690.4688, (25, 17): 1556.2042, (25, 26): 1156.3998, (25, 65): 2065.5852, (25, 55): 1722.0831, (25, 62): 1755.9975, (25, 10): 931.767, (25, 51): 1687.8298, (25, 36): 1044.7532, (25, 45): 436.7936, (25, 34): 942.6043, (25, 18): 981.9302, (25, 28): 1116.509, (25, 30): 1091.9703, (25, 94): 1822.9294, (25, 82): 1809.4475, (25, 19): 1201.5492, (25, 87): 1991.5082, (25, 15): 938.1595, (25, 92): 1052.2152, (25, 72): 1774.2778, (25, 78): 1194.2682, (25, 21): 1794.5901, (25, 85): 1330.3048, (25, 70): 728.9289, (25, 71): 649.1378, (25, 100): 1098.1258, (25, 73): 1029.1388, (25, 5): 1757.3468, (25, 59): 2010.8041, (25, 95): 515.0286, (25, 89): 1837.1653, (25, 86): 1509.1107, (7, 76): 1296.9796, (7, 67): 1758.5807, (7, 77): 888.0907, (7, 84): 1586.4851, (7, 93): 1268.3437, (7, 8): 826.5263, (7, 54): 172.8349, (7, 80): 1682.9092, (7, 16): 1474.1129, (7, 88): 299.772, (7, 42): 1669.7704, (7, 66): 667.6037, (7, 4): 1891.5872, (7, 46): 1516.4977, (7, 98): 1101.0567, (7, 41): 1091.5861, (7, 3): 1111.2466, (7, 31): 1053.5406, (7, 22): 1314.4291, (7, 99): 1048.9005, (7, 39): 1176.8158, (7, 9): 266.3247, (7, 61): 1104.3409, (7, 25): 1037.1401, (7, 6): 1304.3319, (7, 74): 794.6886, (7, 69): 1131.7391, (7, 37): 1135.0234, (7, 32): 280.1038, (7, 68): 1752.8243, (7, 81): 1099.8399, (7, 58): 1248.5633, (7, 47): 1182.4067, (7, 64): 983.1407, (7, 83): 1496.4428, (7, 49): 1783.496, (7, 7): 0.0, (7, 35): 1023.2913, (7, 56): 955.0839, (7, 91): 1670.1529, (7, 27): 763.2175, (7, 79): 924.6661, (7, 44): 1444.7222, (7, 90): 458.7328, (7, 1): 1616.618, (7, 60): 947.1898, (7, 97): 354.7427, (7, 52): 569.3238, (7, 23): 1139.3919, (7, 50): 1638.9636, (7, 40): 1185.91, (7, 12): 1247.9856, (7, 96): 520.9367, (7, 53): 1659.6806, (7, 48): 785.0534, (7, 14): 1760.834, (7, 29): 1752.2929, (7, 43): 373.9315, (7, 2): 1673.2135, (7, 24): 1662.9578, (7, 33): 1418.112, (7, 20): 1924.2748, (7, 57): 488.1678, (7, 75): 938.1772, (7, 13): 1595.5122, (7, 38): 999.8235, (7, 63): 1578.1709, (7, 11): 1195.9757, (7, 17): 1344.4195, (7, 26): 2162.6708, (7, 65): 1853.5908, (7, 55): 1244.4656, (7, 62): 1368.0264, (7, 10): 1557.9703, (7, 51): 1590.037, (7, 36): 1893.9331, (7, 45): 1414.4464, (7, 34): 225.2562, (7, 18): 1534.8742, (7, 28): 1936.9937, (7, 30): 1897.6612, (7, 94): 940.0538, (7, 82): 1603.5452, (7, 19): 2228.4818, (7, 87): 1798.6633, (7, 15): 802.2685, (7, 92): 1877.8853, (7, 72): 1653.8678, (7, 78): 2218.4574, (7, 21): 932.1724, (7, 85): 1242.1531, (7, 70): 1107.2003, (7, 71): 522.4408, (7, 100): 1932.6311, (7, 73): 1692.2158, (7, 5): 1437.9606, (7, 59): 1132.3427, (7, 95): 659.6029, (7, 89): 1550.9313, (7, 86): 1341.1713, (27, 76): 829.5756, (27, 67): 1313.0288, (27, 77): 847.7214, (27, 84): 1200.3441, (27, 93): 712.3248, (27, 8): 797.9865, (27, 54): 749.6817, (27, 80): 1607.3745, (27, 16): 1133.3231, (27, 88): 741.019, (27, 42): 1283.0619, (27, 66): 684.1518, (27, 4): 1472.823, (27, 46): 1012.7501, (27, 98): 1093.971, (27, 41): 948.3245, (27, 3): 487.7269, (27, 31): 638.2719, (27, 22): 825.7513, (27, 99): 980.0284, (27, 39): 731.1507, (27, 9): 534.6014, (27, 61): 657.0543, (27, 25): 655.9696, (27, 6): 759.2137, (27, 74): 988.3785, (27, 69): 1043.5316, (27, 37): 1447.2023, (27, 32): 547.3316, (27, 68): 1624.6406, (27, 81): 495.1017, (27, 58): 632.9564, (27, 47): 987.8404, (27, 64): 865.3451, (27, 83): 1113.1944, (27, 49): 1373.5957, (27, 7): 763.2175, (27, 35): 536.8677, (27, 56): 1112.0001, (27, 91): 1287.7463, (27, 27): 0.0, (27, 79): 729.7315, (27, 44): 1342.2805, (27, 90): 385.7697, (27, 1): 1598.3141, (27, 60): 565.7988, (27, 97): 1078.8093, (27, 52): 449.2903, (27, 23): 1465.6381, (27, 50): 1260.3366, (27, 40): 520.5626, (27, 12): 485.911, (27, 96): 812.8482, (27, 53): 1293.0546, (27, 48): 558.6059, (27, 14): 1320.3875, (27, 29): 1357.0466, (27, 43): 804.4141, (27, 2): 1291.2087, (27, 24): 1280.4074, (27, 33): 1187.9001, (27, 20): 1758.8506, (27, 57): 351.5146, (27, 75): 194.4903, (27, 13): 1583.68, (27, 38): 1239.1747, (27, 63): 1218.7005, (27, 11): 1233.7101, (27, 17): 922.6034, (27, 26): 1760.7815, (27, 65): 1449.9658, (27, 55): 1066.3274, (27, 62): 1102.8418, (27, 10): 1542.8313, (27, 51): 1092.0564, (27, 36): 1700.6294, (27, 45): 1087.2526, (27, 34): 828.0411, (27, 18): 1571.9744, (27, 28): 1771.4618, (27, 30): 1745.6205, (27, 94): 1261.5284, (27, 82): 1187.2901, (27, 19): 1770.8746, (27, 87): 1378.7928, (27, 15): 282.253, (27, 92): 1707.1167, (27, 72): 1175.0127, (27, 78): 1770.1688, (27, 21): 1224.564, (27, 85): 715.9611, (27, 70): 1217.7214, (27, 71): 256.5434, (27, 100): 1753.706, (27, 73): 1655.0696, (27, 5): 1111.5356, (27, 59): 1435.1646, (27, 95): 209.323, (27, 89): 1198.7151, (27, 86): 883.4908, (44, 76): 2161.6544, (44, 67): 2630.0349, (44, 77): 568.0195, (44, 84): 2536.0859, (44, 93): 2023.5427, (44, 8): 2052.7972, (44, 54): 1276.0038, (44, 80): 265.5277, (44, 16): 2474.9157, (44, 88): 1689.9982, (44, 42): 2616.8065, (44, 66): 1898.6404, (44, 4): 2793.0775, (44, 46): 2322.7323, (44, 98): 2366.2132, (44, 41): 2259.0656, (44, 3): 1792.8548, (44, 31): 1980.5287, (44, 22): 2152.6657, (44, 99): 2268.4667, (44, 39): 2070.5863, (44, 9): 1258.9387, (44, 61): 1998.1659, (44, 25): 712.1544, (44, 6): 2070.8559, (44, 74): 685.2615, (44, 69): 2342.7708, (44, 37): 2557.6268, (44, 32): 1225.233, (44, 68): 309.2458, (44, 81): 1809.0651, (44, 58): 1920.3818, (44, 47): 2313.6427, (44, 64): 2162.246, (44, 83): 2451.2131, (44, 49): 2699.5657, (44, 7): 1444.7222, (44, 35): 1877.8333, (44, 56): 2305.4832, (44, 91): 2622.1836, (44, 27): 1342.2805, (44, 79): 2039.4506, (44, 44): 0.0, (44, 90): 1516.7877, (44, 1): 274.893, (44, 60): 1904.0976, (44, 97): 1425.5522, (44, 52): 1666.3668, (44, 23): 2565.9755, (44, 50): 2596.0225, (44, 40): 1787.7023, (44, 12): 1524.0831, (44, 96): 1887.7421, (44, 53): 2629.9799, (44, 48): 1857.7013, (44, 14): 2638.867, (44, 29): 2687.1595, (44, 43): 1777.6061, (44, 2): 2625.642, (44, 24): 2614.9821, (44, 33): 2525.4481, (44, 20): 479.8678, (44, 57): 1162.5486, (44, 75): 1285.1022, (44, 13): 267.6491, (44, 38): 2391.1465, (44, 63): 2557.7936, (44, 11): 301.401, (44, 17): 2259.718, (44, 26): 890.7295, (44, 65): 2775.4256, (44, 55): 2391.6257, (44, 62): 2442.7258, (44, 10): 232.3879, (44, 51): 2399.4905, (44, 36): 456.7039, (44, 45): 426.1213, (44, 34): 1258.1734, (44, 18): 312.6598, (44, 28): 492.5101, (44, 30): 452.983, (44, 94): 2357.9958, (44, 82): 2517.398, (44, 19): 1028.3917, (44, 87): 2701.7527, (44, 15): 1620.7799, (44, 92): 434.7608, (44, 72): 2485.7967, (44, 78): 1005.3379, (44, 21): 2340.0628, (44, 85): 2038.5829, (44, 70): 426.6715, (44, 71): 1257.309, (44, 100): 489.9758, (44, 73): 316.9907, (44, 5): 2453.7313, (44, 59): 2552.1207, (44, 95): 1163.1511, (44, 89): 2538.8257, (44, 86): 2215.3357, (1, 76): 2423.2952, (1, 67): 2896.0118, (1, 77): 783.5476, 
                        (1, 84): 2796.8051, (1, 93): 2289.0378, (1, 8): 2280.1462, (1, 54): 1444.1616, (1, 80): 125.9649, (1, 16): 2731.4614, (1, 88): 1881.8132, (1, 42): 2878.4161, (1, 66): 2121.8184, (1, 4): 3058.6392, (1, 46): 2589.2822, (1, 98): 2593.107, (1, 41): 2497.2537, (1, 3): 2058.0768, (1, 31): 2235.1081, (1, 22): 2415.6779, (1, 99): 2500.446, (1, 39): 2329.069, (1, 9): 1459.7986, (1, 61): 2255.3664, (1, 25): 982.2847, (1, 6): 2336.4528, (1, 74): 824.8242, (1, 69): 2576.9228, (1, 37): 2746.8916, (1, 32): 1424.5274, (1, 68): 248.1248, (1, 81): 2073.2521, (1, 58): 2188.1299, (1, 47): 2556.5896, (1, 64): 2397.4942, (1, 83): 2710.7461, (1, 49): 2963.6825, (1, 7): 1616.618, (1, 35): 2135.1479, (1, 56): 2516.1436, (1, 91): 2883.5236, (1, 27): 1598.3141, (1, 79): 2279.2995, (1, 44): 274.893, (1, 90): 1742.9222, (1, 1): 0.0, (1, 60): 2154.7727, (1, 97): 1541.9874, (1, 52): 1896.6664, (1, 23): 2753.0515, (1, 50): 2856.81, (1, 40): 2056.2786, (1, 12): 1798.1135, (1, 96): 2088.9154, (1, 53): 2890.1957, (1, 48): 2097.2007, (1, 14): 2904.576, (1, 29): 2950.0906, (1, 43): 1968.1342, (1, 2): 2886.9863, (1, 24): 2876.2603, (1, 33): 2773.4861, (1, 20): 405.6677, (1, 57): 1390.8231, (1, 75): 1552.1134, (1, 13): 22.7271, (1, 38): 2591.9477, (1, 63): 2816.711, (1, 11): 420.9487, (1, 17): 2519.5855, (1, 26): 978.9037, (1, 65): 3039.761, (1, 55): 2633.9589, (1, 62): 2692.8492, (1, 10): 59.2257, (1, 51): 2666.5872, (1, 36): 431.7562, (1, 45): 688.2701, (1, 34): 1412.573, (1, 18): 125.7088, (1, 28): 414.509, (1, 30): 368.1669, (1, 94): 2548.8064, (1, 82): 2780.1432, (1, 19): 1138.8667, (1, 87): 2966.6028, (1, 15): 1873.1328, (1, 92): 384.7599, (1, 72): 2752.5779, (1, 78): 1112.0309, (1, 21): 2534.893, (1, 85): 2302.2327, (1, 70): 516.8649, (1, 71): 1492.5934, (1, 100): 431.5723, (1, 73): 78.8607, (1, 5): 2708.9049, (1, 59): 2742.8068, (1, 95): 1410.994, (1, 89): 2796.9758, (1, 86): 2477.1353, (60, 76): 349.99, (60, 67): 825.8328, (60, 77): 1385.515, (60, 84): 675.2549, (60, 93): 348.0556, (60, 8): 377.6997, (60, 54): 1034.3327, (60, 80): 2169.58, (60, 16): 586.7938, (60, 88): 732.5666, (60, 42): 760.6344, (60, 66): 391.8338, (60, 4): 971.9785, (60, 46): 569.3124, (60, 98): 590.0019, (60, 41): 407.647, (60, 3): 317.5506, (60, 31): 106.4782, (60, 22): 367.6154, (60, 99): 467.0477, (60, 39): 229.783, (60, 9): 855.477, (60, 61): 158.2173, (60, 25): 1221.1498, (60, 6): 373.4743, (60, 74): 1475.8689, (60, 69): 511.3163, (60, 37): 1061.6334, (60, 32): 886.8209, (60, 68): 2189.8235, (60, 81): 288.2775, (60, 58): 383.9909, (60, 47): 428.2576, (60, 64): 354.1712, (60, 83): 583.8123, (60, 49): 865.6483, (60, 7): 947.1898, (60, 35): 125.7582, (60, 56): 696.8548, (60, 91): 763.3442, (60, 27): 565.7988, (60, 79): 219.5228, (60, 44): 1904.0976, (60, 90): 513.0407, (60, 1): 2154.7727, (60, 60): 0.0, (60, 97): 1301.2366, (60, 52): 378.0601, (60, 23): 1086.2022, (60, 50): 733.3725, (60, 40): 411.7278, (60, 12): 713.2219, (60, 96): 634.6326, (60, 53): 761.5707, (60, 48): 182.3374, (60, 14): 829.9142, (60, 29): 840.8294, (60, 43): 735.5944, (60, 2): 766.7069, (60, 24): 755.8745, (60, 33): 622.3067, (60, 20): 2324.584, (60, 57): 798.3829, (60, 75): 681.7076, (60, 13): 2139.1497, (60, 38): 845.5363, (60, 63): 682.0387, (60, 11): 1773.8715, (60, 17): 408.3547, (60, 26): 2294.4458, (60, 65): 940.24, (60, 55): 505.9901, (60, 62): 538.6383, (60, 10): 2098.1617, (60, 51): 643.3882, (60, 36): 2265.8831, (60, 45): 1649.049, (60, 34): 1119.3873, (60, 18): 2120.1877, (60, 28): 2337.1816, (60, 30): 2311.4168, (60, 94): 916.7043, (60, 82): 678.3488, (60, 19): 2285.7315, (60, 87): 876.6853, (60, 15): 283.959, (60, 92): 2272.842, (60, 72): 710.4211, (60, 78): 2288.5387, (60, 21): 868.8073, (60, 85): 309.1136, (60, 70): 1741.5581, (60, 71): 683.8322, (60, 100): 2319.2212, (60, 73): 2214.2671, (60, 5): 559.6309, (60, 59): 1045.5669, (60, 95): 743.8649, (60, 89): 658.528, (60, 86): 395.8628, (52, 76): 728.0307, (52, 67): 1197.0346, (52, 77): 1113.2427, (52, 84): 1033.8299, (52, 93): 704.8449, (52, 8): 387.0249, (52, 54): 661.0377, (52, 80): 1929.3162, (52, 16): 930.22, (52, 88): 383.2434, (52, 42): 1118.6578, (52, 66): 243.9339, (52, 4): 1336.7394, (52, 46): 947.3276, (52, 98): 700.1027, (52, 41): 606.9347, (52, 3): 577.1177, (52, 31): 484.5088, (52, 22): 745.1065, (52, 99): 603.7796, (52, 39): 607.8242, (52, 9): 498.8168, (52, 61): 535.0184, (52, 25): 1046.2981, (52, 6): 738.4131, (52, 74): 1155.0635, (52, 69): 680.8409, (52, 37): 999.0946, (52, 32): 533.0932, (52, 68): 1967.3335, (52, 81): 559.3369, (52, 58): 700.5145, (52, 47): 678.9404, (52, 64): 503.0554, (52, 83): 942.4338, (52, 49): 1229.0473, (52, 7): 569.3238, (52, 35): 458.6367, (52, 56): 674.1159, (52, 91): 1120.0852, (52, 27): 449.2903, (52, 79): 399.0685, (52, 44): 1666.3668, (52, 90): 158.4101, (52, 1): 1896.6664, (52, 60): 378.0601, (52, 97): 923.7142, (52, 52): 0.0, (52, 23): 1017.0115, (52, 50): 1089.1944, (52, 40): 664.431, (52, 12): 836.9161, (52, 96): 376.6032, (52, 53): 1113.4128, (52, 48): 228.6724, (52, 14): 1200.1616, (52, 29): 1200.6497, (52, 43): 413.7626, (52, 2): 1123.304, (52, 24): 1112.7133, (52, 33): 911.1207, (52, 20): 2118.1028, (52, 57): 505.8816, (52, 75): 636.2778, (52, 13): 1878.8768, (52, 38): 793.8669, (52, 63): 1031.9519, (52, 11): 1493.267, (52, 17): 780.0833, (52, 26): 2188.6098, (52, 65): 1301.588, (52, 55): 752.4269, (52, 62): 845.0534, (52, 10): 1838.1568, (52, 51): 1021.3547, (52, 36): 2068.3994, (52, 45): 1482.4137, (52, 34): 745.2202, (52, 18): 1845.7363, (52, 28): 2131.0038, (52, 30): 2099.8311, (52, 94): 812.2466, (52, 82): 1044.9166, (52, 19): 2209.3333, (52, 87): 1242.3377, (52, 15): 293.5925, (52, 92): 2067.1894, (52, 72): 1086.9768, (52, 78): 2207.0579, (52, 21): 775.6549, (52, 85): 675.3326, (52, 70): 1442.699, (52, 71): 410.3683, (52, 100): 2117.5835, (52, 73): 1962.8291, (52, 5): 897.0926, (52, 59): 987.4842, (52, 95): 531.2742, (52, 89): 1005.9915, (52, 86): 773.1227, (43, 76): 1065.3901, (43, 67): 1484.6928, (43, 77): 1210.5801, (43, 84): 1299.103, (43, 93): 1082.7545, (43, 8): 500.4799, (43, 54): 540.6861, (43, 80): 2025.855, (43, 16): 1177.6565, (43, 88): 87.8673, (43, 42): 1378.2104, (43, 66): 368.0701, (43, 4): 1601.9885, (43, 46): 1281.8481, (43, 98): 741.5115, (43, 41): 764.6118, (43, 3): 984.9639, (43, 31): 833.7481, (43, 22): 1090.6229, (43, 99): 705.8937, (43, 39): 950.3922, (43, 9): 521.2348, (43, 61): 889.8085, (43, 25): 1274.2881, (43, 6): 1108.9943, (43, 74): 1155.3991, (43, 69): 786.9797, (43, 37): 780.3473, (43, 32): 553.4904, (43, 68): 2086.8259, (43, 81): 964.7277, (43, 58): 1098.3735, (43, 47): 861.8949, (43, 64): 661.0256, (43, 83): 1213.5028, (43, 49): 1495.0039, (43, 7): 373.9315, (43, 35): 842.4521, (43, 56): 581.7029, (43, 91): 1377.0664, (43, 27): 804.4141, (43, 79): 639.6054, (43, 44): 1777.6061, (43, 90): 418.6544, (43, 1): 1968.1342, (43, 60): 735.5944, (43, 97): 660.6628, (43, 52): 413.7626, (43, 23): 788.3695, (43, 50): 1346.1104, (43, 40): 1074.9998, (43, 12): 1241.4787, (43, 96): 155.1128, (43, 53): 1361.6845, (43, 48): 553.6386, (43, 14): 1484.8373, (43, 29): 1459.9178, (43, 43): 0.0, (43, 2): 1379.8384, (43, 24): 1370.2736, (43, 33): 1091.2968, (43, 20): 2253.1665, (43, 57): 679.2765, (43, 75): 998.8622, (43, 13): 1947.7008, (43, 38): 629.1775, (43, 63): 1281.9206, (43, 11): 1547.6143, (43, 17): 1088.4875, (43, 26): 2428.7681, (43, 65): 1560.0208, (43, 55): 914.3879, (43, 62): 1054.2955, (43, 10): 1909.0335, (43, 51): 1347.6939, (43, 36): 2216.0924, (43, 45): 1688.823, (43, 34): 599.1462, (43, 18): 1893.1579, (43, 28): 2266.0493, (43, 30): 2229.0241, (43, 94): 581.2055, (43, 82): 1326.3416, (43, 19): 2474.5019, (43, 87): 1513.1707, (43, 15): 706.7985, (43, 92): 2204.896, (43, 72): 1399.1766, (43, 78): 2468.0355, (43, 21): 567.0537, (43, 85): 1044.64, (43, 70): 1467.0062, (43, 71): 645.2736, (43, 100): 2258.6693, (43, 73): 2041.886, (43, 5): 1139.0309, (43, 59): 775.4253, (43, 95): 798.0893, (43, 89): 1253.7316, (43, 86): 1100.725, (14, 76): 495.5832, (14, 67): 13.2386, (14, 77): 2168.1087, (14, 84): 193.4973, (14, 93): 615.5426, (14, 8): 992.4217, (14, 54): 1861.0136, (14, 80): 2899.5884, (14, 16): 322.1078, (14, 88): 1509.0885, (14, 42): 139.6344, (14, 66): 1118.4622, (14, 4): 154.2507, (14, 46): 317.1861, (14, 98): 892.3129, (14, 41): 752.0495, (14, 3): 846.5004, (14, 31): 726.3245, (14, 22): 495.1009, (14, 99): 841.6309, (14, 39): 608.3235, (14, 9): 1685.3675, (14, 61): 686.5667, (14, 25): 1927.9794, (14, 6): 568.1434, (14, 74): 2289.8708, (14, 69): 785.7156, (14, 37): 1405.6106, (14, 32): 1716.518, (14, 68): 2898.821, (14, 81): 831.9583, (14, 58): 720.9533, (14, 47): 653.3725, (14, 64): 838.2192, (14, 83): 271.8939, (14, 49): 82.8141, (14, 7): 1760.834, (14, 35): 798.1071, (14, 56): 1128.3449, (14, 91): 147.1356, (14, 27): 1320.3875, (14, 79): 846.9771, (14, 44): 2638.867, (14, 90): 1342.4529, (14, 1): 2904.576, (14, 60): 829.9142, (14, 97): 2109.5312, (14, 52): 1200.1616, (14, 23): 1433.8613, (14, 50): 168.1363, (14, 40): 855.4816, (14, 12): 1171.1852, (14, 96): 1345.5987, (14, 53): 180.1305, (14, 48): 976.447, (14, 14): 0.0, (14, 29): 107.6612, (14, 43): 1484.8373, (14, 2): 146.6797, (14, 24): 149.9215, (14, 33): 491.9232, (14, 20): 3009.9481, (14, 57): 1612.8461, (14, 75): 1353.8258, (14, 13): 2891.5685, (14, 38): 1248.9338, (14, 63): 230.5856, (14, 11): 2553.5127, (14, 17): 421.5915, (14, 26): 2841.2038, (14, 65): 144.8483, (14, 55): 634.9229, (14, 62): 471.2057, (14, 10): 2851.1868, (14, 51): 244.1667, (14, 36): 2941.8032, (14, 45): 2315.9104, (14, 34): 1945.3722, (14, 18): 2887.423, (14, 28): 3021.7709, (14, 30): 3003.5611, (14, 94): 1371.1158, (14, 82): 158.6095, (14, 19): 2787.1353, (14, 87): 69.9157, (14, 15): 1072.0506, (14, 92): 2958.813, (14, 72): 157.6798, (14, 78): 2797.8527, (14, 21): 1323.3733, (14, 85): 604.885, (14, 70): 2536.9185, (14, 71): 1496.6368, (14, 100): 2998.6343, (14, 73): 2955.5778, (14, 5): 362.0625, (14, 59): 1387.156, (14, 95): 1524.543, (14, 89): 256.8522, (14, 86): 442.8786, 
                        (2, 76): 463.9876, (2, 67): 155.7624, (2, 77): 2135.6349, (2, 84): 91.7775, (2, 93): 612.6462, (2, 8): 881.5245, (2, 54): 1783.0142, (2, 80): 2888.7762, (2, 16): 202.1919, (2, 88): 1410.2814, (2, 42): 11.8967, (2, 66): 1017.8249, (2, 4): 222.349, (2, 46): 344.3028, (2, 98): 755.9044, (2, 41): 629.8581, (2, 3): 838.0565, (2, 31): 669.2173, (2, 22): 474.667, (2, 99): 714.3601, (2, 39): 561.6326, (2, 9): 1617.1933, (2, 61): 638.3735, (2, 25): 1919.2462, (2, 6): 567.3401, (2, 74): 2241.1036, (2, 69): 653.0861, (2, 37): 1261.7608, (2, 32): 1649.7269, (2, 68): 2894.6021, (2, 81): 819.0266, (2, 58): 729.3475, (2, 47): 529.8578, (2, 64): 723.2983, (2, 83): 182.8947, (2, 49): 117.5281, (2, 7): 1673.2135, (2, 35): 756.567, (2, 56): 993.4326, (2, 91): 3.4636, (2, 27): 1291.2087, (2, 79): 749.9255, (2, 44): 2625.642, (2, 90): 1272.0665, (2, 1): 2886.9863, (2, 60): 766.7069, (2, 97): 2017.0924, (2, 52): 1123.304, (2, 23): 1289.9426, (2, 50): 34.2556, (2, 40): 863.3503, (2, 12): 1198.1299, (2, 96): 1235.892, (2, 53): 33.9369, (2, 48): 895.5343, (2, 14): 146.6797, (2, 29): 80.0809, (2, 43): 1379.8384, (2, 2): 0.0, (2, 24): 10.8815, (2, 33): 349.3745, (2, 20): 3012.9894, (2, 57): 1562.8102, (2, 75): 1345.6873, (2, 13): 2872.9963, (2, 38): 1109.6962, (2, 63): 98.4603, (2, 11): 2523.3658, (2, 17): 369.0225, (2, 26): 2876.585, (2, 65): 180.7739, (2, 55): 502.1579, (2, 62): 340.0267, (2, 10): 2832.2894, (2, 51): 294.0773, (2, 36): 2947.2062, (2, 45): 2320.5941, (2, 34): 1866.149, (2, 18): 2863.1776, (2, 28): 3025.0713, (2, 30): 3004.6557, (2, 94): 1233.2743, (2, 82): 115.8492, (2, 19): 2831.3204, (2, 87): 139.6971, (2, 15): 1028.3218, (2, 92): 2961.5004, (2, 72): 227.9196, (2, 78): 2840.5559, (2, 21): 1186.2332, (2, 85): 589.6951, (2, 70): 2499.4223, (2, 71): 1447.2603, (2, 100): 3003.2919, (2, 73): 2941.121, (2, 5): 241.1845, (2, 59): 1243.3575, (2, 95): 1489.4877, (2, 89): 126.8623, (2, 86): 410.3477, (57, 76): 1117.4925, (57, 67): 1607.0404, (57, 77): 607.5517, (57, 84): 1471.0596, (57, 93): 1027.551, (57, 8): 890.8391, (57, 54): 423.7073, (57, 80): 1424.4487, (57, 16): 1385.0165, (57, 88): 595.2314, (57, 42): 1556.1432, (57, 66): 736.2265, (57, 4): 1760.8602, (57, 46): 1319.3897, (57, 98): 1204.2669, (57, 41): 1108.5558, (57, 3): 817.2135, (57, 31): 893.6042, (57, 22): 1121.3714, (57, 99): 1109.646, (57, 39): 1005.7562, (57, 9): 223.7116, (57, 61): 927.6026, (57, 25): 595.0775, (57, 6): 1072.3498, (57, 74): 678.346, (57, 69): 1186.2244, (57, 37): 1432.4343, (57, 32): 221.1361, (57, 68): 1465.9657, (57, 81): 818.3574, (57, 58): 964.9046, (57, 47): 1173.0857, (57, 64): 1007.2423, (57, 83): 1380.2315, (57, 49): 1656.7951, (57, 7): 488.1678, (57, 35): 814.8863, (57, 56): 1150.0071, (57, 91): 1559.4063, (57, 27): 351.5146, (57, 79): 893.1383, (57, 44): 1162.5486, (57, 90): 354.2392, (57, 1): 1390.8231, (57, 60): 798.3829, (57, 97): 755.2592, (57, 52): 505.8816, (57, 23): 1445.0283, (57, 50): 1529.9447, (57, 40): 863.486, (57, 12): 818.9888, (57, 96): 754.4287, (57, 53): 1559.1636, (57, 48): 711.8815, (57, 14): 1612.8461, (57, 29): 1634.8395, (57, 43): 679.2765, (57, 2): 1562.8102, (57, 24): 1551.9369, (57, 33): 1398.0603, (57, 20): 1621.021, (57, 57): 0.0, (57, 75): 482.9257, (57, 13): 1373.1096, (57, 38): 1247.9404, (57, 63): 1480.1582, (57, 11): 990.0274, (57, 17): 1197.1896, (57, 26): 1750.5659, (57, 65): 1732.5018, (57, 55): 1249.1701, (57, 62): 1323.5513, (57, 10): 1332.3596, (57, 51): 1398.6675, (57, 36): 1575.0628, (57, 45): 1015.6785, (57, 34): 493.5809, (57, 18): 1341.7194, (57, 28): 1633.9618, (57, 30): 1601.1171, (57, 94): 1233.8055, (57, 82): 1468.4432, (57, 19): 1795.4688, (57, 87): 1665.4849, (57, 15): 541.5411, (57, 92): 1570.6827, (57, 72): 1477.1916, (57, 78): 1788.8123, (57, 21): 1208.1666, (57, 85): 1020.7954, (57, 70): 947.0119, (57, 71): 116.2238, (57, 100): 1622.2814, (57, 73): 1456.9981, (57, 5): 1356.8949, (57, 59): 1424.2599, (57, 95): 181.1653, (57, 89): 1456.8761, (57, 86): 1169.9679, (38, 76): 1018.6348, (38, 67): 1254.4181, (38, 77): 1823.2831, (38, 84): 1058.7872, (38, 93): 1129.8295, (38, 8): 473.7823, (38, 54): 1169.2293, (38, 80): 2644.6142, (38, 16): 932.7073, (38, 88): 712.8048, (38, 42): 1113.5756, (38, 66): 555.1366, (38, 4): 1315.546, (38, 46): 1180.8975, (38, 98): 359.582, (38, 41): 543.34, (38, 3): 1163.0398, (38, 31): 886.2148, (38, 22): 1055.2468, (38, 99): 444.1173, (38, 39): 948.0641, (38, 9): 1133.7379, (38, 61): 933.1849, (38, 25): 1827.6375, (38, 6): 1130.2198, (38, 74): 1783.9838, (38, 69): 473.2409, (38, 37): 216.8525, (38, 32): 1168.4139, (38, 68): 2699.4184, (38, 81): 1133.5579, (38, 58): 1208.3409, (38, 47): 623.2457, (38, 64): 518.7166, (38, 83): 1001.3405, (38, 49): 1223.6483, (38, 7): 999.8235, (38, 35): 968.8323, (38, 56): 150.0959, (38, 91): 1108.1629, (38, 27): 1239.1747, (38, 79): 630.2219, (38, 44): 2391.1465, (38, 90): 908.8724, (38, 1): 2591.9477, (38, 60): 845.5363, (38, 97): 1233.5756, (38, 52): 793.8669, (38, 23): 240.73, (38, 50): 1082.4566, (38, 40): 1256.6459, (38, 12): 1544.6477, (38, 96): 503.7198, (38, 53): 1080.5071, (38, 48): 723.4273, (38, 14): 1248.9338, (38, 29): 1181.9038, (38, 43): 629.1775, (38, 2): 1109.6962, (38, 24): 1103.5251, (38, 33): 761.1336, (38, 20): 2861.1965, (38, 57): 1247.9404, (38, 75): 1417.5211, (38, 13): 2571.8987, (38, 38): 0.0, (38, 63): 1018.4055, (38, 11): 2172.5024, (38, 17): 976.8547, (38, 26): 2978.6467, (38, 65): 1266.8955, (38, 55): 618.7131, (38, 62): 780.5838, (38, 10): 2532.7429, (38, 51): 1217.0687, (38, 36): 2819.1305, (38, 45): 2259.3263, (38, 34): 1223.5304, (38, 18): 2520.1729, (38, 28): 2874.1371, (38, 30): 2839.0268, (38, 94): 127.9178, (38, 82): 1110.0794, (38, 19): 3002.895, (38, 87): 1247.9715, (38, 15): 1015.6769, (38, 92): 2811.7594, (38, 72): 1230.2184, (38, 78): 3000.2949, (38, 21): 91.2876, (38, 85): 1077.2669, (38, 70): 2095.0114, (38, 71): 1178.7592, (38, 100): 2864.4815, (38, 73): 2664.552, (38, 5): 894.512, (38, 59): 201.7503, (38, 95): 1316.5834, (38, 89): 992.1762, (38, 86): 1024.2897, (11, 76): 2063.1427, (11, 67): 2545.8986, (11, 77): 388.4729, (11, 84): 2432.0723, (11, 93): 1941.4426, (11, 8): 1872.7478, (11, 54): 1023.6979, (11, 80): 492.8692, (11, 16): 2357.8029, (11, 88): 1461.5088, (11, 42): 2515.5137, (11, 66): 1712.1541, (11, 4): 2706.3153, (11, 46): 2243.0439, (11, 98): 2184.4944, (11, 41): 2098.3617, (11, 3): 1711.8393, (11, 31): 1860.53, (11, 22): 2059.3444, (11, 99): 2095.8793, (11, 39): 1961.8962, (11, 9): 1042.373, (11, 61): 1885.923, (11, 25): 690.4688, (11, 6): 1988.796, (11, 74): 407.4983, (11, 69): 2173.9567, (11, 37): 2326.0216, (11, 32): 1006.7627, (11, 68): 580.1372, (11, 81): 1723.4744, (11, 58): 1849.8651, (11, 47): 2162.6993, (11, 64): 1996.127, (11, 83): 2343.6289, (11, 49): 2607.2958, (11, 7): 1195.9757, (11, 35): 1766.9111, (11, 56): 2099.4773, (11, 91): 2519.9088, (11, 27): 1233.7101, (11, 79): 1883.0814, (11, 44): 301.401, (11, 90): 1337.1249, (11, 1): 420.9487, (11, 60): 1773.8715, (11, 97): 1139.7581, (11, 52): 1493.267, (11, 23): 2332.1157, (11, 50): 2491.9473, (11, 40): 1723.0125, (11, 12): 1507.2338, (11, 96): 1670.0624, (11, 53): 2523.7912, (11, 48): 1701.5589, (11, 14): 2553.5127, (11, 29): 2590.3643, (11, 43): 1547.6143, (11, 2): 2523.3658, (11, 24): 2512.5245, (11, 33): 2385.3467, (11, 20): 758.3622, (11, 57): 990.0274, (11, 75): 1220.54, (11, 13): 400.212, (11, 38): 2172.5024, (11, 63): 2447.6608, (11, 11): 0.0, (11, 17): 2154.349, (11, 26): 1185.0246, (11, 65): 2683.6667, (11, 55): 2239.0519, (11, 62): 2308.1034, (11, 10): 362.0976, (11, 51): 2321.8468, (11, 36): 748.0359, (11, 45): 595.1885, (11, 34): 994.4287, (11, 18): 352.8398, (11, 28): 770.2416, (11, 30): 726.5619, (11, 94): 2128.0623, (11, 82): 2420.7669, (11, 19): 1314.5372, (11, 87): 2612.4428, (11, 15): 1497.5587, (11, 92): 718.762, (11, 72): 2406.1237, (11, 78): 1293.0752, (11, 21): 2114.535, (11, 85): 1948.6502, (11, 70): 125.5709, (11, 71): 1097.4834, (11, 100): 773.8191, (11, 73): 496.2491, (11, 5): 2332.6848, (11, 59): 2322.0102, (11, 95): 1034.2116, (11, 89): 2426.3364, (11, 86): 2117.0308, (17, 76): 107.3919, (17, 67): 417.7141, (17, 77): 1766.6193, (17, 84): 277.805, (17, 93): 278.6842, (17, 8): 618.554, (17, 54): 1440.3239, (17, 80): 2523.5288, (17, 16): 229.4013, (17, 88): 1103.9294, (17, 42): 361.3135, (17, 66): 720.6805, (17, 4): 564.8153, (17, 46): 207.7375, (17, 98): 627.0113, (17, 41): 433.8509, (17, 3): 483.9595, (17, 31): 305.5537, (17, 22): 134.2009, (17, 99): 533.3264, (17, 39): 192.7526, (17, 9): 1263.8314, (17, 61): 270.1215, (17, 25): 1556.2042, (17, 6): 243.4008, (17, 74): 1875.4268, (17, 69): 511.4437, (17, 37): 1169.506, (17, 32): 1295.1032, (17, 68): 2532.2827, (17, 81): 461.4432, (17, 58): 401.5451, (17, 47): 359.0705, (17, 64): 484.8384, (17, 83): 191.5639, (17, 49): 459.7283, (17, 7): 1344.4195, (17, 35): 387.5765, (17, 56): 834.9702, (17, 91): 365.5666, (17, 27): 922.6034, (17, 79): 451.4828, (17, 44): 2259.718, (17, 90): 920.913, (17, 1): 2519.5855, (17, 60): 408.3547, (17, 97): 1695.637, (17, 52): 780.0833, (17, 23): 1197.4898, (17, 50): 337.7665, (17, 40): 526.741, (17, 12): 876.1487, (17, 96): 959.4367, (17, 53): 370.6204, (17, 48): 559.5132, (17, 14): 421.5915, (17, 29): 438.1218, (17, 43): 1088.4875, (17, 2): 369.0225, (17, 24): 358.1775, (17, 33): 375.9354, (17, 20): 2654.6678, (17, 57): 1197.1896, (17, 75): 985.1453, (17, 13): 2505.3625, (17, 38): 976.8547, (17, 63): 298.7141, (17, 11): 2154.349, (17, 17): 0.0, (17, 26): 2549.0497, (17, 65): 535.317, (17, 55): 392.226, (17, 62): 287.4305, (17, 10): 2464.5972, (17, 51): 261.7905, (17, 36): 2590.5415, (17, 45): 1964.7317, (17, 34): 1524.9382, (17, 18): 2494.5323, (17, 28): 2666.8978, (17, 30): 2645.1433, (17, 94): 1084.1229, (17, 82): 271.4591, (17, 19): 2515.7908, (17, 87): 469.3701, (17, 15): 660.0076, (17, 92): 2603.0284, (17, 72): 311.7573, (17, 78): 2522.8845, (17, 21): 1033.7798, (17, 85): 240.4615, (17, 70): 2130.8582, (17, 71): 1081.2516, (17, 100): 2646.0362, (17, 73): 2574.5752, 
                        (17, 5): 223.9097, (17, 59): 1151.1794, (17, 95): 1120.4839, (17, 89): 281.8812, (17, 86): 66.5575, (55, 76): 467.2457, (55, 67): 639.4141, (55, 77): 1854.1818, (55, 84): 442.4438, (55, 93): 622.7723, (55, 8): 418.8167, (55, 54): 1377.3872, (55, 80): 2656.897, (55, 16): 314.8685, (55, 88): 959.1781, (55, 42): 504.3013, (55, 66): 577.6903, (55, 4): 719.0775, (55, 46): 580.4969, (55, 98): 259.3112, (55, 41): 152.9112, (55, 3): 756.4496, (55, 31): 466.4221, (55, 22): 504.2527, (55, 99): 216.4556, (55, 39): 450.9867, (55, 9): 1244.9447, (55, 61): 482.8473, (55, 25): 1722.0831, (55, 6): 604.0528, (55, 74): 1907.4452, (55, 69): 151.0503, (55, 37): 793.3075, (55, 32): 1280.3594, (55, 68): 2683.5585, (55, 81): 726.9004, (55, 58): 733.4854, (55, 47): 78.5077, (55, 64): 261.7138, (55, 83): 382.7668, (55, 49): 619.1898, (55, 7): 1244.4656, (55, 35): 581.0208, (55, 56): 493.4927, (55, 91): 500.1601, (55, 27): 1066.3274, (55, 79): 356.0554, (55, 44): 2391.6257, (55, 90): 910.837, (55, 1): 2633.9589, (55, 60): 505.9901, (55, 97): 1570.5597, (55, 52): 752.4269, (55, 23): 821.662, (55, 50): 472.1151, (55, 40): 830.6554, (55, 12): 1175.115, (55, 96): 762.6647, (55, 53): 476.365, (55, 48): 537.5575, (55, 14): 634.9229, (55, 29): 579.0497, (55, 43): 914.3879, (55, 2): 502.1579, (55, 24): 494.6872, (55, 33): 177.0466, (55, 20): 2823.0675, (55, 57): 1249.1701, (55, 75): 1187.3179, (55, 13): 2617.1854, (55, 38): 618.7131, (55, 63): 406.3686, (55, 11): 2239.0519, (55, 17): 392.226, (55, 26): 2798.4902, (55, 65): 672.5075, (55, 55): 0.0, (55, 62): 163.7188, (55, 10): 2576.232, (55, 51): 605.918, (55, 36): 2766.255, (55, 45): 2152.8926, (55, 34): 1455.2531, (55, 18): 2590.135, (55, 28): 2835.7632, (55, 30): 2808.5847, (55, 94): 738.0668, (55, 82): 491.7156, (55, 19): 2785.2891, (55, 87): 641.8528, (55, 15): 784.7069, (55, 92): 2771.418, (55, 72): 612.4688, (55, 78): 2789.047, (55, 21): 689.6376, (55, 85): 571.0973, (55, 70): 2193.437, (55, 71): 1142.4891, (55, 100): 2818.8128, (55, 73): 2696.8432, (55, 5): 276.1095, (55, 59): 774.752, (55, 95): 1228.9413, (55, 89): 378.8909, (55, 86): 452.5399, (45, 76): 1861.1332, (45, 67): 2305.9799, (45, 77): 572.0505, (45, 84): 2234.3716, (45, 93): 1708.5511, (45, 8): 1863.9411, (45, 54): 1266.2196, (45, 80): 632.0924, (45, 16): 2189.2173, (45, 88): 1602.3143, (45, 42): 2310.8246, (45, 66): 1725.6529, (45, 4): 2469.5118, (45, 46): 1999.0302, (45, 98): 2171.0653, (45, 41): 2034.9986, (45, 3): 1483.1968, (45, 31): 1710.043, (45, 22): 1846.638, (45, 99): 2061.702, (45, 39): 1783.9265, (45, 9): 1177.3575, (45, 61): 1717.7157, (45, 25): 436.7936, (45, 6): 1754.6933, (45, 74): 814.2729, (45, 69): 2128.4513, (45, 37): 2447.7146, (45, 32): 1150.3888, (45, 68): 597.6726, (45, 81): 1503.8622, (45, 58): 1596.0503, (45, 47): 2074.4787, (45, 64): 1948.7256, (45, 83): 2154.9617, (45, 49): 2382.8226, (45, 7): 1414.4464, (45, 35): 1599.5254, (45, 56): 2152.5711, (45, 91): 2317.1873, (45, 27): 1087.2526, (45, 79): 1815.4833, (45, 44): 426.1213, (45, 90): 1351.6552, (45, 1): 688.2701, (45, 60): 1649.049, (45, 97): 1500.7907, (45, 52): 1482.4137, (45, 23): 2460.6055, (45, 50): 2293.4053, (45, 40): 1461.181, (45, 12): 1161.4611, (45, 96): 1770.0373, (45, 53): 2329.1657, (45, 48): 1639.8114, (45, 14): 2315.9104, (45, 29): 2375.322, (45, 43): 1688.823, (45, 2): 2320.5941, (45, 24): 2310.2959, (45, 33): 2267.2151, (45, 20): 694.0806, (45, 57): 1015.6785, (45, 75): 981.0546, (45, 13): 685.8024, (45, 38): 2259.3263, (45, 63): 2262.5682, (45, 11): 595.1885, (45, 17): 1964.7317, (45, 26): 748.2478, (45, 65): 2457.1466, (45, 55): 2152.8926, (45, 62): 2179.0927, (45, 10): 654.6175, (45, 51): 2072.4462, (45, 36): 626.6445, (45, 45): 0.0, (45, 34): 1277.4764, (45, 18): 738.7405, (45, 28): 705.8669, (45, 30): 688.8749, (45, 94): 2249.3823, (45, 82): 2207.9636, (45, 19): 825.5465, (45, 87): 2381.9404, (45, 15): 1368.2018, (45, 92): 643.0741, (45, 72): 2159.399, (45, 78): 812.1223, (45, 21): 2222.9967, (45, 85): 1731.5843, (45, 70): 698.3227, (45, 71): 1081.0941, (45, 100): 683.0048, (45, 73): 708.6874, (45, 5): 2173.105, (45, 59): 2439.2381, (45, 95): 951.3507, (45, 89): 2246.6118, (45, 86): 1913.3896, (28, 76): 2564.2348, (28, 67): 3011.8466, (28, 77): 1055.5743, (28, 84): 2938.247, (28, 93): 2413.6005, (28, 8): 2517.8998, (28, 54): 1768.5029, (28, 80): 295.8212, (28, 16): 2889.6007, (28, 88): 2178.2645, (28, 42): 3015.4207, (28, 66): 2368.5028, (28, 4): 3175.3782, (28, 46): 2704.8617, (28, 98): 2830.2257, (28, 41): 2710.1551, (28, 3): 2187.1327, (28, 31): 2404.4154, (28, 22): 2550.6791, (28, 99): 2727.4008, (28, 39): 2483.6196, (28, 9): 1745.7004, (28, 61): 2415.4055, (28, 25): 1116.509, (28, 6): 2459.9163, (28, 74): 1171.9219, (28, 69): 2798.6948, (28, 37): 3045.0655, (28, 32): 1712.7125, (28, 68): 190.1648, (28, 81): 2207.0664, (28, 58): 2301.748, (28, 47): 2757.2866, (28, 64): 2617.8141, (28, 83): 2857.7244, (28, 49): 3088.4931, (28, 7): 1936.9937, (28, 35): 2296.1078, (28, 56): 2783.1063, (28, 91): 3021.654, (28, 27): 1771.4618, (28, 79): 2489.6527, (28, 44): 492.5101, (28, 90): 1986.6024, (28, 1): 414.509, (28, 60): 2337.1816, (28, 97): 1909.4847, (28, 52): 2131.0038, (28, 23): 3054.1128, (28, 50): 2997.4938, (28, 40): 2166.8987, (28, 12): 1864.2309, (28, 96): 2371.9005, (28, 53): 3033.0025, (28, 48): 2309.7913, (28, 14): 3021.7709, (28, 29): 3080.629, (28, 43): 2266.0493, (28, 2): 3025.0713, (28, 24): 3014.7119, (28, 33): 2958.813, (28, 20): 12.9479, (28, 57): 1633.9618, (28, 75): 1681.7823, (28, 13): 435.6633, (28, 38): 2874.1371, (28, 63): 2965.2844, (28, 11): 770.2416, (28, 17): 2666.8978, (28, 26): 629.0619, (28, 65): 3162.9128, (28, 55): 2835.7632, (28, 62): 2872.5001, (28, 10): 453.7426, (28, 51): 2778.3089, (28, 36): 88.4695, (28, 45): 705.8669, (28, 34): 1750.0587, (28, 18): 539.9639, (28, 28): 0.0, (28, 30): 46.4525, (28, 94): 2845.213, (28, 82): 2912.8995, (28, 19): 801.0389, (28, 87): 3087.7441, (28, 15): 2053.669, (28, 92): 64.3528, (28, 72): 2865.2614, (28, 78): 771.8423, (28, 21): 2825.8138, (28, 85): 2435.589, (28, 70): 887.3531, (28, 71): 1720.729, (28, 100): 34.3507, (28, 73): 345.885, (28, 5): 2872.0699, (28, 59): 3039.0739, (28, 95): 1611.3569, (28, 89): 2948.6641, (28, 86): 2616.8485, (19, 76): 2408.4437, (19, 67): 2775.161, (19, 77): 1395.6333, (19, 84): 2756.5459, (19, 93): 2238.3404, (19, 8): 2568.4667, (19, 54): 2086.7174, (19, 80): 1016.2513, (19, 16): 2744.677, (19, 88): 2390.6805, (19, 42): 2820.1038, (19, 66): 2451.0709, (19, 4): 2931.109, (19, 46): 2488.0518, (19, 98): 2857.1484, (19, 41): 2692.1355, (19, 3): 2043.5697, (19, 31): 2319.7944, (19, 22): 2384.1645, (19, 99): 2738.7144, (19, 39): 2361.9656, (19, 9): 1980.2485, (19, 61): 2310.9178, (19, 25): 1201.5492, (19, 6): 2278.3197, (19, 74): 1622.9054, (19, 69): 2793.7947, (19, 37): 3204.0386, (19, 32): 1956.5016, (19, 68): 898.0836, (19, 81): 2070.7381, (19, 58): 2115.7102, (19, 47): 2709.814, (19, 64): 2622.5929, (19, 83): 2692.3563, (19, 49): 2864.8134, (19, 7): 2228.4818, (19, 35): 2204.8627, (19, 56): 2881.1933, (19, 91): 2828.199, (19, 27): 1770.8746, (19, 79): 2483.1545, (19, 44): 1028.3917, (19, 90): 2100.8676, (19, 1): 1138.8667, (19, 60): 2285.7315, (19, 97): 2326.0563, (19, 52): 2209.3333, (19, 23): 3219.9609, (19, 50): 2810.5899, (19, 40): 1989.3518, (19, 12): 1640.0786, (19, 96): 2536.5866, (19, 53): 2848.189, (19, 48): 2323.9856, (19, 14): 2787.1353, (19, 29): 2868.6581, (19, 43): 2474.5019, (19, 2): 2831.3204, (19, 24): 2822.3147, (19, 33): 2865.5611, (19, 20): 801.2671, (19, 57): 1795.4688, (19, 75): 1611.2613, (19, 13): 1153.3234, (19, 38): 3002.895, (19, 63): 2796.1754, (19, 11): 1314.5372, (19, 17): 2515.7908, (19, 26): 173.3549, (19, 65): 2931.8028, (19, 55): 2785.2891, (19, 62): 2772.9308, (19, 10): 1150.1703, (19, 51): 2548.7273, (19, 36): 737.1119, (19, 45): 825.5465, (19, 34): 2101.65, (19, 18): 1254.0969, (19, 28): 801.0389, (19, 30): 833.4655, (19, 94): 3010.5146, (19, 82): 2715.5489, (19, 19): 0.0, (19, 87): 2856.9346, (19, 15): 2028.8192, (19, 92): 790.5639, (19, 72): 2631.6603, (19, 78): 29.7227, (19, 21): 2978.6667, (19, 85): 2275.7327, (19, 70): 1437.8199, (19, 71): 1838.2088, (19, 100): 768.6566, (19, 73): 1096.2557, (19, 5): 2738.5425, (19, 59): 3193.5416, (19, 95): 1691.6184, (19, 89): 2786.876, (19, 86): 2454.3884, (15, 76): 576.4733, (15, 67): 1066.0077, (15, 77): 1109.1819, (15, 84): 936.6512, (15, 93): 492.4603, (15, 8): 550.6599, (15, 54): 848.5742, (15, 80): 1886.1989, (15, 16): 860.5126, (15, 88): 669.9434, (15, 42): 1020.9996, (15, 66): 472.2955, (15, 4): 1221.1727, (15, 46): 778.6057, (15, 98): 828.3456, (15, 41): 670.856, (15, 3): 309.2874, (15, 31): 363.1061, (15, 22): 579.8325, (15, 99): 710.2905, (15, 39): 467.3757, (15, 9): 645.9453, (15, 61): 389.9976, (15, 25): 938.1595, (15, 6): 535.3683, (15, 74): 1217.8208, (15, 69): 768.9518, (15, 37): 1230.403, (15, 32): 671.7808, (15, 68): 1905.8654, (15, 81): 299.9718, (15, 58): 450.4457, (15, 47): 706.2794, (15, 64): 594.3122, (15, 83): 847.1548, (15, 49): 1118.2439, (15, 7): 802.2685, (15, 35): 274.0445, (15, 56): 877.5381, (15, 91): 1024.8817, (15, 27): 282.253, (15, 79): 455.6658, (15, 44): 1620.7799, (15, 90): 344.5317, (15, 1): 1873.1328, (15, 60): 283.959, (15, 97): 1151.6014, (15, 52): 293.5925, (15, 23): 1251.6288, (15, 50): 996.241, (15, 40): 385.391, (15, 12): 550.3107, (15, 96): 662.5822, (15, 53): 1027.2278, (15, 48): 298.8658, (15, 14): 1072.0506, (15, 29): 1098.0557, (15, 43): 706.7985, (15, 2): 1028.3218, (15, 24): 1017.4456, (15, 33): 906.2656, (15, 20): 2041.0495, (15, 57): 541.5411, (15, 75): 417.6547, (15, 13): 1857.8311, (15, 38): 1015.6769, (15, 63): 950.3431, (15, 11): 1497.5587, (15, 17): 660.0076, (15, 26): 2028.3554, (15, 65): 1194.3089, (15, 55): 784.7069, (15, 62): 822.1517, (15, 10): 1816.8725, (15, 51): 857.7041, (15, 36): 1982.8651, (15, 45): 1368.2018, (15, 34): 933.8409, (15, 18): 1841.3736, (15, 28): 2053.669, (15, 30): 2027.6597, (15, 94): 1058.2616, (15, 82): 930.0642, (15, 19): 2028.8192, 
                        (15, 87): 1125.9768, (15, 15): 0.0, (15, 92): 1989.3214, (15, 72): 935.6925, (15, 78): 2029.9269, (15, 21): 1015.9661, (15, 85): 480.9679, (15, 70): 1471.1102, (15, 71): 425.3745, (15, 100): 2035.9584, (15, 73): 1931.7674, (15, 5): 836.428, (15, 59): 1216.5431, (15, 95): 463.8851, (15, 89): 928.7996, (15, 86): 629.3353}
    return customers, warehouses, plants, products, customer_demands, plant_wh_distance, wh_cust_distance

In [24]:
def optimal_location(number_of_whs, warehouses, customers, plants, products, plant_product_info, customer_demands, distance_band, scenario_name):
    
    start_time=time.time() 
    
    #problem is initialized
    jade_problem = LpProblem("JADE", LpMinimize)
    
    #adding flow variables from plant to warehouse
    flow_vars_pw = LpVariable.dicts("flow_pw", [(p, w, k) for p in plants for w in warehouses for k in products 
                                                if (p,k) in plant_product_info.keys() ],lowBound=0,cat='Continuous')
    #adding flow variables from warehouse to customer
    flow_vars_wc = LpVariable.dicts("flow_wc", [(w, c, k) for w in warehouses for c in customers for k in products if customer_demands[c,k]>0],
                                                lowBound=0,upBound = 1,cat='Integer')
    #adding facility binary variable if a warehouse is opened or not
    facility_vars = LpVariable.dicts("facility_vars", [w for w in warehouses], lowBound=0, upBound = 1, cat='Integer')
    
    #adding single source constraints for warehouse to customer.  This will be across all products
    single_source_wc = LpVariable.dicts("single_source_wc", [(w, c) for w in warehouses for c in customers],
                                                lowBound=0,upBound = 1,cat='Integer')
    

    #constraints
    #every customer should be served by a warehouse
    for c in customers:
        for k in products:
            if customer_demands[c,k]>0:
                jade_problem += LpConstraint(e = lpSum([flow_vars_wc[w, c, k] for w in warehouses]),
                                     sense = LpConstraintEQ,
                                     name = str(c)+'_'+str(k)+"_Served",
                                     rhs = 1)

    #flow conservation at warehouses
    for w in warehouses:
        for k in products:
            jade_problem += LpConstraint(e = lpSum([flow_vars_wc[w, c, k]*customer_demands[c, k] for c in customers if customer_demands[c,k]>0]) - lpSum([flow_vars_pw[p, w, k] for p in plants if (p,k) in plant_product_info.keys()]),
                                     sense = LpConstraintEQ,
                                     name = str(w)+'_'+str(k)+"_FlowConstraint",
                                     rhs = 0)


    #capacity constraint
    for (p,k), value in plant_product_info.items():
        jade_problem += LpConstraint(e = lpSum([flow_vars_pw[p, w, k] for w in warehouses]),
                                    sense = LpConstraintLE,
                                    name = str(p)+'_'+str(k)+"_plantproductConstraint",
                                    rhs = plant_product_info[p, k])
        
    #if there is a flow from a warehouse, it should be opened    
    for w in warehouses:
        jade_problem += LpConstraint(e = lpSum([flow_vars_wc[w, c, k] for c in customers for k in products if customer_demands[c, k]>0]) - facility_vars[w]*10000000,
                                    sense = LpConstraintLE,
                                    name = str(w)+"_facilityopenconstraint",
                                    rhs = 0)
    
    #restrict the number of open warehouses
    jade_problem += LpConstraint(e = lpSum([facility_vars[w] for w in warehouses]),
                                     sense = LpConstraintEQ,
                                     name = "numberofwhsconstraints",
                                     rhs = number_of_whs)

    # Use the user's status to fix some warehouses in the solution
   
    for facility in warehouses:
        w = facility
        jade_problem += LpConstraint(e = facility_vars[facility] , 
                                   sense=LpConstraintGE, 
                                    name=str(facility) + "_" + "Lower Bound",
                                    rhs=wh_status[w][1])
    for facility in warehouses:
        w = facility
        jade_problem += LpConstraint(e = facility_vars[facility] , 
                                   sense=LpConstraintLE, 
                                    name=str(facility) + "_" + "Upper Bound",
                                    rhs=wh_status[w][2])
    
    #Set the single constraints in two steps.  First tie the single source to the flow variables. 
    for w in warehouses:
        for c in customers:
            for k in products:
                if customer_demands[c,k]>0:  
                    jade_problem += LpConstraint(e = flow_vars_wc[w, c, k]-single_source_wc[w,c],
                                             sense = LpConstraintLE,
                                             name = str(w)+'_'+str(c)+'_'+str(k)+"_Single Source Check",
                                             rhs =0)

    # Second, we need to prevent two warehouses from serving the same customer
    for c in customers:
        jade_problem += LpConstraint(e = lpSum([single_source_wc[w,c] for w in warehouses]),
                                     sense = LpConstraintLE,
                                     name = str(c)+"_Single Source Constraint",
                                     rhs = 1)

    
    #Create Inound and Outbound Costs to account for minimum charges
    p_w_cost = {}
    for p in plants:
        for w in warehouses:
            p_w_cost[p,w] = ic_trans_cost*plant_wh_distance[p,w]
            if plant_wh_distance[p,w]<= ic_min_trans/ic_trans_cost :
                p_w_cost[p,w]= ic_min_trans
    
    w_c_cost = {}
    for w in warehouses:
        for c in customers:
            w_c_cost[w,c] = ob_trans_cost*wh_cust_distance[w, c]
            if wh_cust_distance[w, c] <= ob_min_trans/ob_trans_cost:  #this division is now in miles units
                w_c_cost[w,c]= ob_min_trans
    
    #setting objective function
#     total_weighted_demand_objective = lpSum([flow_vars_pw[p, w, k]*ic_trans_cost*plant_wh_distance[p,w] for p in plants for w in warehouses for k in products if (p,k) in plant_product_info.keys()]) + lpSum([flow_vars_wc[w, c, k]*wh_cust_distance[w, c]*customer_demands[c, k]*ob_trans_cost for w in warehouses for c in customers for k in products])
    total_weighted_demand_objective = lpSum([flow_vars_pw[p, w, k]*p_w_cost[p,w] for p in plants for w in warehouses for k in products if (p,k) in plant_product_info.keys()]) + lpSum([flow_vars_wc[w, c, k]*w_c_cost[w,c]*customer_demands[c, k] for w in warehouses for c in customers for k in products])


    jade_problem.setObjective(total_weighted_demand_objective)

    jade_problem.solve()

    total_flow_pw = {(p, w, k): flow_vars_pw[p, w, k].varValue for p in plants for w in warehouses for k in products if (p,k) in plant_product_info.keys()  and flow_vars_pw[p, w, k].varValue > 0}

    total_flow_wc = {(w, c, k): flow_vars_wc[w, c, k].varValue for w in warehouses for c in customers for k in products if flow_vars_wc[w, c, k].varValue>0}

    print('Status:'+ LpStatus[jade_problem.status])
    print("Objective: "+ str(jade_problem.objective.value()))
    file.write('\nStatus:'+ LpStatus[jade_problem.status])
    
    total_demand = sum(customer_demands.values())
    file.write("\nTotal Demand:"+ str(total_demand))
    
    file.write("\nObjective: "+ str(jade_problem.objective.value()))
        
    end_time = time.time()
    
    time_diff = end_time - start_time
    
    file.write("\nRun Time in seconds {:.1f}" .format(time_diff))
    print("Run Time in seconds {:.1f}" .format(time_diff))
    
    #preparing data to write in excel sheets
    opened_warehouses = []
    
    warehouse_list = []
    
    for w in facility_vars.keys():
        if facility_vars[w].varValue >0:
            warehouse_list.append(w)
            
    list_warehouses_open = set(list(warehouse_list))
    
    total_demand_to_warehouse = {w: sum(customer_demands[c, k]*flow_vars_wc[w, c, k].varValue for c in customers)
                                    for w in list_warehouses_open for k in products}
    
    for w in list_warehouses_open:
        wh = {
                'Warehouse Key': w,
                'Warehouse City':warehouses[w][3],
                'State':warehouses[w][4],
                'ZipCode':warehouses[w][5],
                'Lat':warehouses[w][7],
                'Lon':warehouses[w][8],
                'Total Demand to Warehouse':total_demand_to_warehouse[w]
        }
        opened_warehouses.append(wh)
    
    
            
    df_wh = pd.DataFrame.from_records(opened_warehouses)
    
    df_wh = df_wh[['Warehouse Key', 'Warehouse City', 'State', 'ZipCode', 'Total Demand to Warehouse']]
            
    # writing detailed files
    writer = pd.ExcelWriter(scenario_name+'_detailed.xlsx')
    df_wh.to_excel(writer,'Opened Warehouses',index=False)
    
    customers_assignment = []   
    maps_assign_wc = {}
    for (w, c, k) in total_flow_wc.keys():
            cust = {
                'Warehouse':str(warehouses[w][3]+','+warehouses[w][4]),
                'Customer':str(customers[c][2]+','+customers[c][3]),
                'Product': str(k),
                'Customer Demand': customer_demands[c, k],
                'Distance': wh_cust_distance[w,c],
                'Warehouse Latitude' : warehouses[w][7],
                'Warehouse Longitude' : warehouses[w][8],
                'Customers Latitude' : customers[c][6],
                'Customers Longitude': customers[c][7]
            }
            customers_assignment.append(cust)
            if (w, c) not in maps_assign_wc:
                maps_assign_wc[w, c] = 1
                
    plants_assignment = []
    maps_assign_pw = {}
    for (p, w, k) in total_flow_pw.keys():
            plant = {
                'Warehouse':str(warehouses[w][3]+','+warehouses[w][4]),
                'Plant':str(plants[p][1]+','+plants[p][2]),
                'Product': str(k),
                'Quantity': total_flow_pw[p, w, k],
                'Distance': plant_wh_distance[p, w],
                'Warehouse Latitude' : warehouses[w][7],
                'Warehouse Longitude' : warehouses[w][8],
                'Plant Latitude' : plants[p][5],
                'Plant Longitude': plants[p][6]
            }
            plants_assignment.append(plant)
            if (p, w) not in maps_assign_pw:
                maps_assign_pw[p, w] = 1             
                
    df_cu = pd.DataFrame.from_records(customers_assignment)
    df_cu_copy = df_cu.copy()
    df_cu = df_cu[['Warehouse', 'Customer', 'Product', 'Distance', 'Customer Demand']]
    df_cu.to_excel(writer,'Customers Assignment',index = False)
      
    df_pl = pd.DataFrame.from_records(plants_assignment)
    df_pl_copy = df_pl.copy()
    df_pl = df_pl[['Plant', 'Warehouse', 'Product','Distance', 'Quantity']]
    df_pl.to_excel(writer,'Plants to Warehouse',index = False)
    
    writer.close()
    
    distance_band_1 = distance_band[0]
    distance_band_2 = distance_band[1]
    distance_band_3 = distance_band[2]
    distance_band_4 = distance_band[3]
    #writing percent demand within each distance bands
    total_demand = sum(df_cu['Customer Demand'])
    percent_demand_distance_band_1 = sum(df_cu[df_cu['Distance']<distance_band_1]['Customer Demand'])*100/total_demand
    percent_demand_distance_band_2 = sum(df_cu[df_cu['Distance']<distance_band_2]['Customer Demand'])*100/total_demand
    percent_demand_distance_band_3 = sum(df_cu[df_cu['Distance']<distance_band_3]['Customer Demand'])*100/total_demand
    percent_demand_distance_band_4 = sum(df_cu[df_cu['Distance']<distance_band_4]['Customer Demand'])*100/total_demand
    file.write("\nPercent Demand served within {} miles : {:.1f}" .format(distance_band[0], percent_demand_distance_band_1))
    file.write("\nPercent Demand served within {} miles : {:.1f}" .format(distance_band[1], percent_demand_distance_band_2))
    file.write("\nPercent Demand served within {} miles : {:.1f}" .format(distance_band[2], percent_demand_distance_band_3))
    file.write("\nPercent Demand served within {} miles : {:.1f}" .format(distance_band[3], percent_demand_distance_band_4))
    
    return df_pl_copy, df_cu_copy, list_warehouses_open
 

In [25]:
# In this function, we are visualizaing the input data-- the data before the optimization has run
def input_visual(warehouses, customers, plants):
    warehouse_list = []
    for w in warehouses.keys():
        wh = {
                    'text':'Warehouse-'+warehouses[w][0],
                    'State':warehouses[w][4],
                    'ZipCode':warehouses[w][5],
                    'lat':warehouses[w][7],
                    'long':warehouses[w][8],
                    'cnt':10000000,
                    'size' : 20,
                    'color' : 'rgba(0, 100, 0)'
                    }
        warehouse_list.append(wh)    

    customer_list =[] 
    for c in customers.keys():
        cust = {
            'text':'Customer-'+customers[c][0],
            'State':customers[c][4],
            'ZipCode':customers[c][5],
            'lat':customers[c][6],
            'long':customers[c][7] ,
            'cnt':10000000,
            'size' : 3,
            'color' : 'rgb(255, 0, 0)'
        }
        customer_list.append(cust)  
        
    plant_list = []
    
    
    for p in plants.keys():
        pl = {
            'text':'Plant-'+plants[p][1],
            'State':plants[p][2],
            'ZipCode':plants[p][3],
            'lat':plants[p][5],
            'long':plants[p][6] ,
            'cnt':10000000,
            'size' : 20,
            'color' : 'rgb(0, 0, 255)'            
        }
        plant_list.append(pl)

    df = pd.DataFrame.from_records(warehouse_list)
    df['shape'] =  "triangle-down"
    df_cust = pd.DataFrame.from_records(customer_list)
    df_cust['shape'] =  "circle"
    df_pl = pd.DataFrame.from_records(plant_list)
    df_pl['shape'] = "square"
    
    df = df.append(df_cust, ignore_index = True) 
    df = df.append(df_pl, ignore_index = True)
    
    list_dict = {'warehouses':df, 'customers':df_cust, 'plants':df_pl}
    
    locations=[]
    for key, i in list_dict.items():
        location = dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = i['long'],
        lat = i['lat'],
        text =i['text'],
        marker = dict(
            size = i['size'],
            # sizeref = 2. * max(df_sub['pop']/scale) / (25 ** 2),
            symbol = i['shape'],
            color = i['color'],
            line = dict(
                    width=3,
                    color='rgba(68, 68, 68, 0)'           
        )),
        name = key)
        locations.append(location)
    
    
    

    layout = dict(
            title = 'JADE - Input',
            showlegend = True, 
            geo = dict(
                scope='usa',
                projection=dict( type='albers usa' ),
                showland = True,
                landcolor = 'rgb(243, 243, 243)',
                countrycolor = 'rgb(204, 204, 204)',
            ),
        )
    py.offline.plot({ "data":locations, "layout":layout}, filename = scenario_name+'_input.html')  
    

In [26]:
# In this function, we are visualizaing the output data-- the data after the optimization has run
def output_visual(warehouses, customers, plants, df_cu, df_pl, wh_loc, scenario_name ):   
    warehouse_list = []
    for w in wh_loc:
            wh = {
                        'text':'Warehouse-'+warehouses[w][0],
                        'State':warehouses[w][4],
                        'ZipCode':warehouses[w][5],
                        'lat':warehouses[w][7],
                        'long':warehouses[w][8],
                        'cnt':10000000,
                        'size' : 20,
                        'color' : 'rgba(0, 100, 0)'
                        }
            warehouse_list.append(wh)    

    customer_list =[] 
    for c in customers.keys():
        cust = {
            'text':'Customer-'+customers[c][0],
            'State':customers[c][4],
            'ZipCode':customers[c][5],
            'lat':customers[c][6],
            'long':customers[c][7] ,
            'cnt':10000000,
            'size' : 3,
            'color' : 'rgb(255, 0, 0)'
        }
        customer_list.append(cust)


    plant_list = []  
    for p in plants.keys():
        pl = {
            'text':'Plant-'+plants[p][1],
            'State':plants[p][2],
            'ZipCode':plants[p][3],
            'lat':plants[p][5],
            'long':plants[p][6] ,
            'cnt':10000000,
            'size' : 30,
            'color' : 'rgb(0, 0, 255)'            
        }
        plant_list.append(pl)

    df = pd.DataFrame.from_records(warehouse_list)
    df['shape'] =  "triangle-down"
    df_cust = pd.DataFrame.from_records(customer_list)
    df_cust['shape'] =  "circle"
    df = df.append(df_cust, ignore_index = True) 
    df_plants = pd.DataFrame.from_records(plant_list)
    df_plants['shape'] = "square"

    df_cu['lane'] = df_cu.apply(lambda x: str(x['Warehouse'])+'-'+str(x['Customer'])+','+'Product'+str(x['Product']),axis=1)

    df_pl['lane'] = df_pl.apply(lambda x: str(x['Plant'])+'-'+str(x['Warehouse']),axis=1)
#     df = df.append(df_plants, ignore_index = True)



    list_dict = {'warehouses':df, 'customers':df_cust}

    outbound_locations=[]
    for key, i in list_dict.items():
        location = dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = i['long'],
        lat = i['lat'],
        text =i['text'],
        marker = dict(
            size = i['size'],
            # sizeref = 2. * max(df_sub['pop']/scale) / (25 ** 2),
            symbol = i['shape'],
            color = i['color'],
            line = dict(
                    width=3,
                    color='rgba(68, 68, 68, 0)'           
        )),
        name = key)
        outbound_locations.append(location)

    inbound_paths = []
    outbound_paths = []

    for i in range( len( df_cu) ):
            outbound_paths.append(
                dict(
                    type = 'scattergeo',
                    locationmode = 'USA-states',
                    lon = [ df_cu['Warehouse Longitude'][i], df_cu['Customers Longitude'][i] ],
                    lat = [ df_cu['Warehouse Latitude'][i], df_cu['Customers Latitude'][i] ],
                    mode = 'lines',
                    line = dict(
                        width = 1,
                        color = 'red',
                    ),
                    opacity = 0.8,
                    name = df_cu['lane'][i]
                )
            )  

    layout = dict(
            title = 'JADE - Outbound Lanes',
            showlegend = True, 
            geo = dict(
                scope='usa',
                projection=dict( type='albers usa' ),
                showland = True,
                landcolor = 'rgb(243, 243, 243)',
                countrycolor = 'rgb(204, 204, 204)',
            ),
        )

    py.offline.plot({"data":outbound_locations+outbound_paths, "layout":layout},filename=scenario_name+'_outbound.html')

    for i in range( len( df_pl) ):
            inbound_paths.append(
                dict(
                    type = 'scattergeo',
                    locationmode = 'USA-states',
                    lon = [ df_pl['Plant Longitude'][i], df_pl['Warehouse Longitude'][i] ],
                    lat = [ df_pl['Plant Latitude'][i], df_pl['Warehouse Latitude'][i] ],
                    mode = 'lines',
                    line = dict(
                        width = 4,
                        color = 'purple',
                    ),
                    opacity = 0.8,
                    name = df_pl['lane'][i]
                )
            )  


    layout = dict(
            title = 'JADE - Inbound Lanes',
            showlegend = True, 
            geo = dict(
                scope='usa',
                projection=dict( type='albers usa' ),
                showland = True,
                landcolor = 'rgb(243, 243, 243)',
                countrycolor = 'rgb(204, 204, 204)',
            ),
        )
    
    list_dict = {'plants':df_plants}
    plant_locations = []
    for key, i in list_dict.items():
        location = dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = i['long'],
        lat = i['lat'],
        text =i['text'],
        marker = dict(
            size = i['size'],
            # sizeref = 2. * max(df_sub['pop']/scale) / (25 ** 2),
            symbol = i['shape'],
            color = i['color'],
            line = dict(
                    width=3,
                    color='rgba(68, 68, 68, 0)'           
        )),
        name = key)
        plant_locations.append(location)
    
    

    py.offline.plot({"data":plant_locations+outbound_locations+inbound_paths, "layout":layout},filename=scenario_name+'_inbound.html')


    layout = dict(
            title = 'JADE - Output Lanes',
            showlegend = True, 
            geo = dict(
                scope='usa',
                projection=dict( type='albers usa' ),
                showland = True,
                landcolor = 'rgb(243, 243, 243)',
                countrycolor = 'rgb(204, 204, 204)',
            ),
        )



    py.offline.plot({"data":outbound_locations+outbound_paths+inbound_paths+plant_locations, "layout":layout},filename=scenario_name+'_output.html')

In [27]:
def test_input(warehouses, customers, plants, products, customer_demands, plant_wh_distance, wh_cust_distance, distance_band, number_of_whs, plant_product_info):
    for c in customers:
        for w in warehouses:
            if(wh_cust_distance[w, c]>=0):
                pass
            else:
                file.write(f'\nDistance between warehouse {w} and customer {c} is not available or invalid')
    
    for p in plants:
        for w in warehouses:
            if(plant_wh_distance[p, w]>=0):
                pass
            else:
                file.write(f'\nDistance between warehouse {p} and customer {w} is not available or invalid')

    for k in products:
        for c in customers:
            if(customer_demands[c, k]>=0):
                pass
            else:
                file.write(f'\nDemand for Customer {c} is not available')
        
    
    if ((distance_band[0] < distance_band[1]) and (distance_band[1]  < distance_band[2]) and (distance_band[2]  < distance_band[3]) ):
        pass
    else:
        file.write(f'\nDistance bands are not in ascending order')
        
    if isinstance(number_of_whs, int)==False:
        file.write(f'\nnumber_of_whs = {number_of_whs} is not integer')
    
    for k in products:
        demand = sum(customer_demands[c, k] for c in customers if customer_demands[c, k]>0)
        capacity = sum(plant_product_info[p, k] for p in plants if (p, k) in plant_product_info.keys())
        if demand > capacity:
            file.write(f'\nDemand for product {k} is less than capacity')
 
   
              

In [28]:
distance_band = [distance_band_1, distance_band_2, distance_band_3, distance_band_4]

print("Running - this may take 10-30 seconds.")
file = open(scenario_name+'_summary' + '.txt',"w")

file.write('Getting Input Data')
customers, warehouses, plants, products, customer_demands, plant_wh_distance, wh_cust_distance = get_data()

for k in products:
    a = ''
    for p in plants:
        if (p,k) in plant_product_info.keys():
            if plant_product_info[p, k] > 0:
                a = a + (str(p) + " ")
    a = a.replace(" ", ",")        
    file.write('\nProduct ' + str(k) +' can be made in plant '+ a[:-1])            
    
file.write('\nTesting Input Data')
test_input(warehouses, customers, plants, products, customer_demands, plant_wh_distance, wh_cust_distance, distance_band, number_of_whs, plant_product_info)

file.write('\nBuilding the Model')
df_pl, df_cu, wh_loc = optimal_location(number_of_whs, warehouses, customers, plants, products, plant_product_info, customer_demands,distance_band, scenario_name)

print('If status is infeasible check the summary file for data issues')
if input_map == True:
    input_visual(warehouses, customers, plants)

if output_map == True:
    output_visual(warehouses, customers, plants, df_cu, df_pl, wh_loc, scenario_name)
file.close()

Running - this may take 10-30 seconds.


KeyboardInterrupt: 